In [1]:
import cmomy.options as options

In [2]:
options.set_options(nmax=8)

In [3]:
from importlib import reload

In [4]:
import numpy as np

import cmomy.central as central
import cmomy.accumulator as accumulator

In [6]:
x = np.random.rand(10, 11, 12)

In [15]:
import cmomy.xcentral as xcentral
reload(central)
reload(xcentral)

<module 'cmomy.xcentral' from '/Users/wpk/Documents/python/cmomy/xcentral.py'>

In [16]:
def _get_comom(w, x, y, moments, axis=0, broadcast=True):
    if w.ndim == 1 and w.ndim != x.ndim and len(w) == x.shape[axis]:
        shape = [1] * x.ndim
        shape[axis] = -1
        w = w.reshape(*shape)

    if w.shape != x.shape:
        w = np.broadcast_to(w, x.shape)

    if y.ndim != x.ndim and y.ndim == 1 and len(y) == x.shape[axis]:
        shape = [1] * x.ndim
        shape[axis] = -1
        y = y.reshape(*shape)

    if broadcast and y.shape != x.shape:
        y = np.broadcast_to(y, x.shape)

    
    shape = list(x.shape)
    shape.pop(axis)
    shape = tuple(shape) + tuple(x+1 for x in moments)
    
    out = np.zeros(shape)
    wsum = w.sum(axis)
    wsum_inv = 1.0 / wsum

    wsum_keep = w.sum(axis, keepdims=True)
    wsum_keep_inv = 1.0 / wsum_keep

    xave = (w * x).sum(axis, keepdims=True) * wsum_keep_inv
    dx = x - xave

    yave = (w * y).sum(axis, keepdims=True) * wsum_keep_inv
    dy = (y - yave)
    


    for i in range(moments[0] + 1):
        for j in range(moments[1] + 1):
            if i ==0 and j==0:
                val = wsum

            elif i + j == 1:
                val = (w * x**i * y**j).sum(axis) * wsum_inv

            else:
                val = (w * dx**i * dy**j).sum(axis) * wsum_inv
                
            out[..., i, j] = val

    return out


In [52]:
import pytest

In [ ]:
pytest.fixture()

In [53]:
x = np.random.rand(2, 3, 4)
idx = np.random.choice(3, (10, 3))

In [57]:
np.moveaxis(x, 2, 0).shape

(4, 2, 3)

In [55]:
np.take(x, idx, axis=1).shape

(2, 10, 3, 4)

In [17]:
x = np.random.rand(100)
y = np.random.rand(100)
w = np.random.rand(100)

In [18]:
a = central.central_comoments(x, y, (3,3), w=w, broadcast=True)

In [36]:
b = _get_comom(w, x, y, (3,1), broadcast=True)

In [45]:
mom = 1

In [51]:
list(zip(*zip(list('abc'), list('xyz'))))

[('a', 'b', 'c'), ('x', 'y', 'z')]

In [48]:
list(range(1, 3))

[1, 2]

In [46]:
if len(mom) == 1:
    print('hello')

TypeError: object of type 'int' has no len()

In [37]:
s = central.StatsAccumCov.from_vals((x, y), w=w, mom=(3,1), broadcast=True)

In [44]:
hasattr(central.StatsAccum, 'push_stat')

True

In [42]:
for a, *x in zip(list('abc'), [1,2,3], list('xyz')):
    print('a',a, 'x', x, type(x))

a a x [1, 'x'] <class 'list'>
a b x [2, 'y'] <class 'list'>
a c x [3, 'z'] <class 'list'>


In [39]:
np.testing.assert_allclose(b, s.data)

In [23]:
ss = central.StatsAccum.from_vals(x, w=w, mom=(3,), broadcast=True)

In [31]:
c = central.StatsAccum.from_stat(ss.mean(), ss.data[2:], ss.weight(), mom=ss.mom)

In [35]:
a = np.random.rand(1, 3)
b = np.random.rand(1, 3)
np.array((a,b)).shape

(2, 1, 3)

In [33]:
c.data - ss.data

array([0., 0., 0., 0.])

In [26]:
ss.shape_mom_var

(2,)

IndexError: index 2 is out of bounds for axis 1 with size 2

In [24]:
all([x > 0 for x in s.mom])

False

In [15]:
ss = central.StatsAccum.from_vals(x, w=w, mom=3)

In [19]:
s.weight()

50.14161217493299

In [21]:
s.data[...,0] - ss.data

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.73472348e-18])

In [16]:
s.data[...,0]

array([5.01416122e+01, 4.74942999e-01, 7.83656146e-02, 3.41644250e-03])

In [17]:
ss.data

array([5.01416122e+01, 4.74942999e-01, 7.83656146e-02, 3.41644250e-03])

In [156]:
s.weight()

49.59576777970217

In [157]:
ss.weight()

49.59576777970217

In [158]:
s.data

array([[ 4.95957678e+01],
       [ 5.17675481e-01],
       [ 8.22005608e-02],
       [-6.89442358e-03]])

In [159]:
ss.data

array([ 4.95957678e+01,  5.49896292e-01,  7.95650648e-02, -5.91686420e-03])

In [141]:
ss.data - b[...,0]

array([[ 0.00000000e+00,  5.55111512e-17, -2.77555756e-17,
        -2.21177243e-17],
       [ 0.00000000e+00,  0.00000000e+00, -2.77555756e-17,
        -3.03576608e-17],
       [ 0.00000000e+00,  1.11022302e-16, -1.38777878e-17,
        -1.08420217e-17],
       [ 0.00000000e+00, -2.77555756e-16, -2.77555756e-17,
         5.37764278e-17],
       [ 0.00000000e+00,  1.11022302e-16,  1.38777878e-17,
         4.27717757e-17],
       [ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17,
         2.29850861e-17],
       [ 0.00000000e+00, -2.22044605e-16, -2.77555756e-17,
         1.06685494e-16],
       [ 0.00000000e+00,  3.33066907e-16, -1.38777878e-16,
        -6.24500451e-17],
       [ 0.00000000e+00, -1.11022302e-16,  2.77555756e-17,
         1.56125113e-17],
       [ 0.00000000e+00,  1.11022302e-16,  0.00000000e+00,
        -6.93889390e-18]])

In [139]:
s.shape_mom

(4, 1)

In [135]:
s.data

array([[[ 4.52433041e+01],
        [ 4.88844981e-01],
        [ 7.53246960e-02],
        [ 2.99006581e-03]],

       [[ 4.52433041e+01],
        [ 5.16242991e-01],
        [ 8.02770573e-02],
        [-4.51055359e-04]],

       [[ 4.52433041e+01],
        [ 5.30577695e-01],
        [ 8.14340297e-02],
        [-5.95161516e-03]],

       [[ 4.52433041e+01],
        [ 5.11986195e-01],
        [ 9.12517635e-02],
        [-6.97139026e-03]],

       [[ 4.52433041e+01],
        [ 5.22907849e-01],
        [ 9.01263338e-02],
        [-2.18953917e-03]],

       [[ 4.52433041e+01],
        [ 5.01614178e-01],
        [ 9.66084899e-02],
        [-2.82834683e-03]],

       [[ 4.52433041e+01],
        [ 5.38698933e-01],
        [ 9.00124520e-02],
        [-1.75714153e-03]],

       [[ 4.52433041e+01],
        [ 5.30750110e-01],
        [ 1.05432968e-01],
        [ 5.38400533e-04]],

       [[ 4.52433041e+01],
        [ 4.87077287e-01],
        [ 9.50745370e-02],
        [ 4.40251205e-03]],

       [[ 

In [120]:
_get_cmom_vec(w, x, 3) - b[...,0]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [115]:
np.testing.assert_allclose(a, b)

In [61]:
a

array([[ 5.23880218e+01,  4.44322196e-01,  9.40364269e-02,
         4.33686837e-03],
       [ 4.77749088e-01, -1.44289061e-02, -3.84347218e-04,
        -1.91821039e-03],
       [ 8.71252692e-02, -2.53172616e-03,  7.81257720e-03,
         8.69718766e-05],
       [ 3.27387711e-03, -2.06826487e-03,  2.34706551e-04,
        -2.31534648e-04]])

In [62]:
b

array([[ 5.23880218e+01,  4.44322196e-01,  9.40364269e-02,
         4.33686837e-03],
       [ 4.77749088e-01,  5.23880218e+01, -3.84347218e-04,
        -1.91821039e-03],
       [ 8.71252692e-02, -2.53172616e-03,  5.23880218e+01,
         8.69718766e-05],
       [ 3.27387711e-03, -2.06826487e-03,  2.34706551e-04,
         5.23880218e+01]])

In [29]:
x, w = _get_data_single(100, weighted=False)
x.shape, w.shape

((100,), (100,))

In [31]:
x, w = _get_data_vec((100,), weighted=False)
x.shape, w.shape

((100,), (100,))

In [32]:
_get_cmom_single(x, w, 2)

array([1.00000000e+02, 5.38467306e-01, 6.87958455e-02])

In [33]:
_get_cmom_vec(x, w, 2)

array([1.00000000e+02, 5.38467306e-01, 6.87958455e-02])

In [6]:
x0 = np.random.rand(100, 10)
x1 = np.random.rand(100, 10)



In [12]:
s = central.StatsAccum.from_vals(x0)

In [22]:
xs = xcentral.xStatsAccum.zeros(mom=2, shape=10, dims_mom='mom')

In [23]:
xs.values.sum()

<xarray.DataArray (dim_0: 10, mom: 3)>
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
Dimensions without coordinates: dim_0, mom

In [ ]:
xs.data.sum()

In [268]:
xs.push_vals(x0)

In [285]:
xcentral.xcentral_moments(xr.DataArray(x0), 2, )

<xarray.DataArray (dim_1: 10, mom_0: 3)>
array([[1.00000000e+02, 4.67710622e-01, 8.10839508e-02],
       [1.00000000e+02, 4.79324262e-01, 7.66613978e-02],
       [1.00000000e+02, 4.46403110e-01, 7.78677829e-02],
       [1.00000000e+02, 4.82415329e-01, 9.13403277e-02],
       [1.00000000e+02, 5.12064774e-01, 7.09031179e-02],
       [1.00000000e+02, 4.94945766e-01, 7.83040040e-02],
       [1.00000000e+02, 5.38072802e-01, 8.07837481e-02],
       [1.00000000e+02, 5.36416779e-01, 7.99748788e-02],
       [1.00000000e+02, 4.68908693e-01, 8.84212022e-02],
       [1.00000000e+02, 5.39913076e-01, 8.08576523e-02]])
Dimensions without coordinates: dim_1, mom_0

In [283]:
xcentral.xStatsAccum.zeros(mom=2).values

<xarray.DataArray (mom_0: 3)>
array([0., 0., 0.])
Dimensions without coordinates: mom_0

In [279]:
xcentral.xcentral_moments(x0, 2,  dims_mom='mom')

AssertionError: 

In [271]:
s.values

array([[1.00000000e+02, 4.67710622e-01, 8.10839508e-02],
       [1.00000000e+02, 4.79324262e-01, 7.66613978e-02],
       [1.00000000e+02, 4.46403110e-01, 7.78677829e-02],
       [1.00000000e+02, 4.82415329e-01, 9.13403277e-02],
       [1.00000000e+02, 5.12064774e-01, 7.09031179e-02],
       [1.00000000e+02, 4.94945766e-01, 7.83040040e-02],
       [1.00000000e+02, 5.38072802e-01, 8.07837481e-02],
       [1.00000000e+02, 5.36416779e-01, 7.99748788e-02],
       [1.00000000e+02, 4.68908693e-01, 8.84212022e-02],
       [1.00000000e+02, 5.39913076e-01, 8.08576523e-02]])

In [272]:
xs.values - s.values

<xarray.DataArray (dim_0: 10, mom: 3)>
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
Dimensions without coordinates: dim_0, mom

<xarray.DataArray (dim_0: 10, dim_1: 3)>
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
Dimensions without coordinates: dim_0, dim_1

In [136]:
out = central.central_comoments(x0, x1, mom=(3,3))

In [189]:
a = central.StatsAccum.from_vals(x0[:2, :], mom=(3,))

In [193]:
b = central.StatsAccum.zeros(shape=10, mom=3).push_val(x0[0]).push_val(x0[1])

In [198]:
import xarray as xr

In [218]:
class A:
    def __init__(self, data):
        assert isinstance(data, np.ndarray)
        self.data = data
            
    def add(self, val):
        return type(self)(self.data + val)
        
class B(A):
    def __init__(self, data):
#        assert isinstance(data, xr.DataArray)
        self.xdata = data
        print(type(data), type(data.data))
        super().__init__(data=data.data)
        
        
    def madd(self, val):
        return super(B, self).add(val)
    
    


In [219]:
x = np.zeros(3)

a = A(x)

a.add(2).add(2).data

array([4., 4., 4.])

In [226]:
d = xr.DataArray(xr.DataArray(x, dims='a'), dims='b')

In [ ]:
d.copy()

In [220]:
b = B(xr.DataArray(x))

In [221]:
b.add(2)

AssertionError: 

array([2., 2., 2.])

In [206]:
a.data

array([0., 0., 0.])

In [140]:
x0[:, 0].shape

(100,)

In [144]:
shape = list(x0[:, 0].shape)

In [145]:
shape.pop(0)

100

In [152]:
s0 = central.StatsAccum.from_vals(x0[:,:], axis=0)

In [158]:
tuple(np.eye(2, dtype=np.int))

(array([1, 0]), array([0, 1]))

In [153]:
s0._single_index(3)

(Ellipsis, 3)

In [168]:
s = central.StatsAccumCov.from_vals((x0[:,0], x1[:,0]), mom=(4,4))

In [ ]:
# mean((x - mean(x)) * (y - mean(y)))
# = mean()

In [183]:
np.sum?

In [181]:
np.random.rand(100).fill(0)

In [180]:
np.diag_indices(2)

(array([0, 1]), array([0, 1]))

In [176]:
(1, 0)

(1, 0)

In [172]:
s.data[(...,) + tuple(np.array([[1,2], [2,3]]))]

array([ 0.00259471, -0.00019283])

In [174]:
s.data[..., 1, 2]

array(0.00259471)

In [175]:
s.data[..., 2, 3]

array(-0.00019283)

In [166]:
s.data[(...,) + tuple(np.eye(2, dtype=np.int)*1)]

array([0.44966227, 0.53387787])

In [163]:
s.data[s._single_index(2)]

array([0.08766535, 0.0975826 ])

In [128]:
s.data[(...,0,1)]

array(0.53387787)

In [111]:
s = central.StatsAccumCov.zeros((3,3), shape=(10,))

In [114]:
s = central.StatsAccumCov.from_vals((x0, x1), mom=(3,3))

In [121]:
s.data.shape

(10, 4, 4)

In [ ]:
np.full_like()

In [118]:
np.asarray(s)

array([[[ 1.00000000e+02,  5.33877874e-01,  9.75825975e-02,
         -2.83397238e-03],
        [ 4.49662271e-01, -7.03660110e-03,  2.93057749e-03,
         -1.64107699e-03],
        [ 8.76653467e-02, -1.59031905e-03,  9.29691052e-03,
         -6.07358665e-04],
        [ 3.63772327e-03, -1.83240734e-03,  5.94688202e-04,
         -4.31412251e-04]],

       [[ 1.00000000e+02,  5.39306557e-01,  7.66925541e-02,
         -3.68079900e-03],
        [ 5.07675527e-01,  1.39120629e-02, -5.01713423e-03,
          3.02944334e-03],
        [ 8.58977827e-02, -1.29655732e-03,  7.93050598e-03,
         -6.87620165e-04],
        [-1.28738539e-03,  3.10690620e-03, -1.01129591e-03,
          6.35978649e-04]],

       [[ 1.00000000e+02,  4.77931192e-01,  8.03854138e-02,
          5.54653190e-04],
        [ 5.18160961e-01,  2.45162133e-03,  6.39771207e-04,
          3.42131459e-04],
        [ 8.11836565e-02, -3.06843325e-03,  7.44305774e-03,
         -5.51095936e-04],
        [-3.40086007e-03, -3.25217898e-

In [117]:
s.data.fill()

array([[[ 1.00000000e+02,  5.33877874e-01,  9.75825975e-02,
         -2.83397238e-03],
        [ 4.49662271e-01, -7.03660110e-03,  2.93057749e-03,
         -1.64107699e-03],
        [ 8.76653467e-02, -1.59031905e-03,  9.29691052e-03,
         -6.07358665e-04],
        [ 3.63772327e-03, -1.83240734e-03,  5.94688202e-04,
         -4.31412251e-04]],

       [[ 1.00000000e+02,  5.39306557e-01,  7.66925541e-02,
         -3.68079900e-03],
        [ 5.07675527e-01,  1.39120629e-02, -5.01713423e-03,
          3.02944334e-03],
        [ 8.58977827e-02, -1.29655732e-03,  7.93050598e-03,
         -6.87620165e-04],
        [-1.28738539e-03,  3.10690620e-03, -1.01129591e-03,
          6.35978649e-04]],

       [[ 1.00000000e+02,  4.77931192e-01,  8.03854138e-02,
          5.54653190e-04],
        [ 5.18160961e-01,  2.45162133e-03,  6.39771207e-04,
          3.42131459e-04],
        [ 8.11836565e-02, -3.06843325e-03,  7.44305774e-03,
         -5.51095936e-04],
        [-3.40086007e-03, -3.25217898e-

In [116]:
s.data - out

array([[[ 0.00000000e+00,  2.22044605e-16,  6.93889390e-17,
         -8.23993651e-18],
        [-3.33066907e-16,  2.60208521e-18,  1.64798730e-17,
          8.67361738e-19],
        [ 2.77555756e-17, -3.03576608e-18,  3.46944695e-18,
         -1.40946282e-18],
        [ 3.12250226e-17, -4.33680869e-19,  3.68628739e-18,
         -7.04731412e-19]],

       [[ 0.00000000e+00,  0.00000000e+00,  4.16333634e-17,
          2.86229374e-17],
        [ 1.11022302e-16,  3.46944695e-18, -1.99493200e-17,
          3.46944695e-18],
        [ 1.38777878e-17,  3.25260652e-18,  3.46944695e-18,
          1.08420217e-18],
        [-7.04731412e-17,  4.33680869e-19, -5.85469173e-18,
         -2.16840434e-19]],

       [[ 0.00000000e+00, -1.11022302e-16,  2.77555756e-17,
          9.75781955e-19],
        [ 2.22044605e-16,  1.30104261e-18,  8.67361738e-19,
          2.16840434e-19],
        [-2.77555756e-17, -8.67361738e-19, -4.33680869e-18,
         -2.16840434e-18],
        [-7.80625564e-18,  2.16840434e-

In [112]:
s.push_vals((x0, x1))

In [113]:
s.data - out

array([[[ 0.00000000e+00,  2.22044605e-16,  6.93889390e-17,
         -8.23993651e-18],
        [-3.33066907e-16,  2.60208521e-18,  1.64798730e-17,
          8.67361738e-19],
        [ 2.77555756e-17, -3.03576608e-18,  3.46944695e-18,
         -1.40946282e-18],
        [ 3.12250226e-17, -4.33680869e-19,  3.68628739e-18,
         -7.04731412e-19]],

       [[ 0.00000000e+00,  0.00000000e+00,  4.16333634e-17,
          2.86229374e-17],
        [ 1.11022302e-16,  3.46944695e-18, -1.99493200e-17,
          3.46944695e-18],
        [ 1.38777878e-17,  3.25260652e-18,  3.46944695e-18,
          1.08420217e-18],
        [-7.04731412e-17,  4.33680869e-19, -5.85469173e-18,
         -2.16840434e-19]],

       [[ 0.00000000e+00, -1.11022302e-16,  2.77555756e-17,
          9.75781955e-19],
        [ 2.22044605e-16,  1.30104261e-18,  8.67361738e-19,
          2.16840434e-19],
        [-2.77555756e-17, -8.67361738e-19, -4.33680869e-18,
         -2.16840434e-18],
        [-7.80625564e-18,  2.16840434e-

In [91]:
ys

[array([0.7714728 , 0.72589147, 0.11108316, 0.43381615, 0.63451052,
        0.73487299, 0.55198069, 0.87559912, 0.16066365, 0.93269434,
        0.29577426, 0.23878501, 0.69286884, 0.10181239, 0.33817416,
        0.87157702, 0.68788842, 0.683344  , 0.02903054, 0.83783242,
        0.27246259, 0.66693962, 0.39102639, 0.41241505, 0.28837328,
        0.57917685, 0.14536265, 0.99022826, 0.51096603, 0.83648796,
        0.05624031, 0.6748751 , 0.79862409, 0.93325238, 0.95866962,
        0.61727482, 0.26660916, 0.20198787, 0.51764741, 0.24364867,
        0.65849364, 0.01540544, 0.16768223, 0.96586946, 0.01515308,
        0.16195565, 0.75972865, 0.51540148, 0.35189139, 0.93564643,
        0.37617332, 0.74003929, 0.13901492, 0.58912859, 0.3577573 ,
        0.34750318, 0.57373574, 0.37269552, 0.79500611, 0.07421627,
        0.20356574, 0.17412094, 0.4828841 , 0.27698684, 0.45343968,
        0.33225952, 0.54438992, 0.5125404 , 0.91495881, 0.71058382,
        0.43909444, 0.12582905, 0.25329527, 0.39

In [130]:
x = np.random.rand(100, 2)
y = np.random.rand(*x.shape)

z = np.concatenate((x, y), axis=0)

In [133]:
o = central.StatsAccum.zeros(mom=2, shape=2)

o.push_vals(x)
o.push_vals(y)

In [134]:
o.mean() - z.mean(0)

array([-1.11022302e-16,  4.44089210e-16])

In [135]:
oo = central.StatsAccum.from_vals(x).push_vals(y)

In [138]:
ooo = central.StatsAccum.from_data(oo.data)

In [ ]:
np.cov()

In [ ]:
np.average()

In [ ]:
np.zeros

In [33]:
xr, target = o.check_vals(x, axis=0, target='vals')

In [37]:
wr = o.check_weights(None, target, 0)

In [43]:
f = o._push.vals

In [ ]:
o._data

In [50]:
o._data_flat

array([[1.00000000e+02, 4.58445838e-01, 8.94095818e-02]])

In [48]:
f(o._data, wr, xr)

In [13]:
o.shape

()

In [6]:
from importlib import reload
reload(central)
reload(accumulator)

<module 'cmomy.accumulator' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator.py'>

In [7]:
import cmomy.statsarray as statsarray
reload(statsarray)

<module 'cmomy.statsarray' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/statsarray.py'>

In [8]:
import xarray as xr

In [9]:
x = np.random.rand(100, 20, 3)
y = np.random.rand(100)
w = np.random.rand(100, 20, 3)

In [352]:
import cmomy.xcentral as xcentral
reload(xcentral)
reload(central)

<module 'cmomy.central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/central.py'>

In [353]:
sx = xcentral.xStatsAccum(2, shape=(2,3), dims=['x','y'])
s  = central.StatsAccum(2, shape=(2,3))

In [354]:
xx = xr.DataArray(x, dims=['rec','x','y']).transpose('x',...)
yy = xr.DataArray(y, dims=['rec'])
xw = xr.DataArray(w, dims=['rec','x','y']).transpose('y',...)

In [355]:
sx = xcentral.xStatsAccum.from_vals(xx, xw, axis='rec')
s = central.StatsAccum.from_vals(x, w, axis=0)

In [364]:
cx = xcentral.xStatsAccumCov.from_vals(xx, yy, broadcast=True, w=xw, axis='rec')
c = central.StatsAccumCov.from_vals(x, y, broadcast=True, w=w)

In [406]:
tmp = cx.values.copy()

In [407]:
tmp[..., 0, 0] = 0

In [ ]:
np.zeros()

In [405]:
cx._single_index_selector

<bound method xStatsAccumBase._single_index_selector of <cmomy.xcentral.xStatsAccumCov object at 0x7fbe787c56d0>>

In [402]:
tmp.loc[cx._single_index_selector(1, select=False)]  = np.nan

In [404]:
tmp

<xarray.DataArray (x: 20, y: 3, mom_0: 3, mom_1: 3)>
array([[[[ 5.29136572e+01,             nan,  7.92109044e-02],
         [            nan, -3.02362251e-03, -9.91166441e-04],
         [ 8.36634445e-02, -4.49528158e-04,  6.40161038e-03]],

        [[ 5.48910598e+01,             nan,  8.65083123e-02],
         [            nan,  1.04918964e-02, -1.75977104e-03],
         [ 6.97789286e-02, -2.76127419e-03,  7.04239504e-03]],

        [[ 4.59537550e+01,             nan,  9.28736090e-02],
         [            nan, -6.54385997e-06,  3.09848662e-03],
         [ 7.37742635e-02,  1.52028613e-03,  7.19282732e-03]]],


       [[[ 5.12608875e+01,             nan,  8.68118787e-02],
         [            nan, -8.59207212e-03, -1.53663767e-03],
         [ 8.99198070e-02,  2.38559284e-03,  8.97741864e-03]],

        [[ 5.11347796e+01,             nan,  8.74590822e-02],
         [            nan, -9.76043317e-03,  4.89186451e-03],
         [ 7.75633643e-02, -3.14839046e-03,  6.92770097e-03]],
...
        [[ 5.09835132e+01,             nan,  8.77605507e-02],
         [            nan,  6.51167107e-03,  4.74991856e-03],
         [ 7.02645300e-02,  2.65453105e-03,  4.90990993e-03]],

        [[ 4.96761325e+01,             nan,  8.03152818e-02],
         [            nan, -1.86885724e-03, -4.23554388e-03],
         [ 6.80766241e-02, -7.92216410e-04,  4.79905673e-03]]],


       [[[ 5.21827963e+01,             nan,  8.60240514e-02],
         [            nan, -1.43602644e-03,  6.77261407e-04],
         [ 6.47661886e-02, -7.42874260e-03,  7.08509836e-03]],

        [[ 5.27243106e+01,             nan,  7.63500877e-02],
         [            nan,  3.76517216e-03, -1.89526688e-03],
         [ 7.09984185e-02, -1.96780629e-03,  6.24803315e-03]],

        [[ 4.87864590e+01,             nan,  8.42865004e-02],
         [            nan, -2.04745789e-02, -3.33454239e-04],
         [ 7.21643332e-02,  7.91875036e-04,  6.17994355e-03]]]])
Dimensions without coordinates: x, y, mom_0, mom_1

In [396]:
cx._single_index_selector(1, select=False)

{'mom_0': <xarray.DataArray (variable: 2)>
 array([1, 0])
 Dimensions without coordinates: variable,
 'mom_1': <xarray.DataArray (variable: 2)>
 array([0, 1])
 Dimensions without coordinates: variable}

In [393]:
cx._single_index_selector(2, select=False)

{'mom_0': <xarray.DataArray (variable: 2)>
 array([2, 0])
 Dimensions without coordinates: variable,
 'mom_1': <xarray.DataArray (variable: 2)>
 array([0, 2])
 Dimensions without coordinates: variable}

In [379]:
sx.moments

(2,)

In [383]:
sx._moments_shape_var

(1,)

In [388]:
sx._data_shape

(20, 3, 3)

In [389]:
sx._data.shape

(20, 3, 3)

In [387]:
sx._shape_var

(20, 3, 1)

In [386]:
cx._shape_var

(20, 3, 1, 1)

In [385]:
cx._shape_var

(20, 3, 1, 1)

In [384]:
sx._shape_var

(20, 3, 1)

In [382]:
sx.data.shape

(20, 3, 3)

In [380]:
sx._moments_shape

(3,)

In [376]:
%timeit -n 1 -r 1 a = central.central_comoments(x, y, broadcast=True, moments=(3,3))

1.58 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [378]:
%timeit -n 1 -r 1 b = xcentral.xcentral_comoments(xx, yy, broadcast=True, moments=(3,3), axis='rec')

8.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [375]:
np.testing.assert_allclose(a,b)

In [ ]:
npc.mean()

In [368]:
np.testing.assert_allclose(cx.mean(),c.mean())

In [367]:
np.testing.assert_allclose(cx.var(),c.var())

In [308]:
cx.var() - c.var()

<xarray.DataArray (x: 20, y: 3, variable: 2)>
array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

...

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])
Coordinates:
  * variable  (variable) <U5 'mom_0' 'mom_1'
Dimensions without coordinates: x, y

In [290]:
cx.mean() - c.mean()

<xarray.DataArray (x: 20, y: 3, variable: 2)>
array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

...

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])
Coordinates:
  * variable  (variable) <U5 'mom_0' 'mom_1'
Dimensions without coordinates: x, y

In [287]:
cx.cmom() - c.cmom()

<xarray.DataArray (x: 20, y: 3, mom_0: 3, mom_1: 3)>
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
...
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])
Dimensions without coordinates: x, y, mom_0, mom_1

In [282]:
cx.data.shape

(20, 3, 3, 3)

In [280]:
c.data.shape

(20, 3, 3, 3)

In [272]:
c.values - cx.data

ValueError: operands could not be broadcast together with shapes (20,3,3,3) (100,3,3,3) 

In [176]:
sx3 = xcentral.xStatsAccum.from_vals(x=xx, w=xw, axis='rec')

In [180]:
freq = central.randsamp_freq(nrep=10, size=100)

In [ ]:
a = central.StatsAccum.from_resample_vals(x=x, w=w, freq=freq)

b = central.StatsAccum.from_resample_vals(x=np.swapaxes(x, 1, 0), w=np.swapaxes(w, 1, 0), axis=1, freq=freq)


c1 = xcentral.xStatsAccum.from_resample_vals(x=xx.transpose('rec','x','y').values, 
                                            w=xw.transpose('rec','x','y').values, 
                                            freq=freq, axis=0, dims=('x','y'))

In [204]:
%%timeit -n 1 -r 1
c2 = xcentral.xStatsAccum.from_resample_vals(x=xx,
                                            w=xw,
                                            freq=freq, axis='rec')

1.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [203]:
%%timeit -n 1 -r 1
c3 = xcentral.xcentral_moments(x=xx, weights=xfreq * xw, axis='rec', moments=2)

11.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [199]:
c2.values - c3

<xarray.DataArray (rep: 10, x: 20, y: 3, mom_0: 3)>
array([[[[ 0.00000000e+00, -1.11022302e-16,  4.16333634e-17],
         [ 0.00000000e+00, -5.55111512e-17,  0.00000000e+00],
         [ 0.00000000e+00, -1.11022302e-16,  4.16333634e-17]],

        [[ 0.00000000e+00, -3.33066907e-16, -6.93889390e-17],
         [ 0.00000000e+00,  5.55111512e-17,  0.00000000e+00],
         [ 0.00000000e+00,  5.55111512e-17,  6.93889390e-17]],

        [[ 0.00000000e+00,  0.00000000e+00,  2.77555756e-17],
         [ 0.00000000e+00, -5.55111512e-16, -5.55111512e-17],
         [ 0.00000000e+00, -1.11022302e-16, -6.93889390e-17]],

        ...,

        [[ 0.00000000e+00, -4.44089210e-16, -1.38777878e-17],
         [ 0.00000000e+00,  0.00000000e+00, -2.77555756e-17],
         [ 0.00000000e+00,  1.66533454e-16, -6.93889390e-18]],

        [[ 0.00000000e+00, -1.11022302e-16, -6.93889390e-17],
         [ 0.00000000e+00,  2.22044605e-16,  4.16333634e-17],
...
         [ 0.00000000e+00, -1.11022302e-16, -5.55111512e-17],
         [ 0.00000000e+00, -1.66533454e-16,  5.55111512e-17]],

        [[ 0.00000000e+00,  2.77555756e-16,  1.38777878e-17],
         [ 0.00000000e+00, -3.33066907e-16, -4.16333634e-17],
         [ 0.00000000e+00,  2.77555756e-16,  1.38777878e-17]],

        ...,

        [[ 0.00000000e+00, -1.11022302e-16,  1.38777878e-17],
         [ 0.00000000e+00,  0.00000000e+00, -1.11022302e-16],
         [ 0.00000000e+00,  1.11022302e-16,  1.38777878e-17]],

        [[ 0.00000000e+00, -5.55111512e-17,  0.00000000e+00],
         [ 0.00000000e+00,  1.11022302e-16,  1.38777878e-17],
         [ 0.00000000e+00, -1.11022302e-16, -1.38777878e-17]],

        [[ 0.00000000e+00, -1.11022302e-16, -9.02056208e-17],
         [ 0.00000000e+00,  0.00000000e+00, -4.16333634e-17],
         [ 0.00000000e+00, -2.22044605e-16,  1.38777878e-17]]]])
Dimensions without coordinates: rep, x, y, mom_0

In [186]:
xfreq = xr.DataArray(freq, dims=['rep','rec'])


<xarray.DataArray (rep: 10, rec: 100, y: 3, x: 20)>
array([[[[0.08278816, 0.79663081, 0.13270858, ..., 0.1362653 ,
          0.37091097, 0.07059638],
         [0.74916803, 0.70188727, 0.65244172, ..., 0.85327151,
          0.13892098, 0.28649396],
         [0.47598476, 0.07559265, 0.38410816, ..., 0.27864158,
          0.5126352 , 0.21477218]],

        [[1.30478086, 0.3042921 , 0.87112678, ..., 1.98358642,
          1.01279488, 0.08701104],
         [1.20141826, 1.04653307, 0.66224135, ..., 1.17625878,
          1.91053076, 0.71756884],
         [0.83468085, 0.98534232, 0.54745309, ..., 1.60789124,
          0.98708095, 0.46045611]],

        [[0.89724049, 0.3835192 , 0.49722408, ..., 0.54709012,
          0.9384066 , 0.17582491],
         [0.06389944, 0.70751191, 0.99245071, ..., 0.98833476,
          0.19527106, 0.4746848 ],
         [0.03348814, 0.42060564, 0.91413859, ..., 0.90049252,
          0.88103879, 0.97549848]],
...
        [[1.37138933, 0.27348005, 0.17424553, ..., 2.58621993,
          0.72884847, 1.74600816],
         [2.70394992, 1.11087282, 1.30203823, ..., 0.43727103,
          1.24286493, 1.06466311],
         [2.65762175, 1.18326343, 2.50840152, ..., 0.24035563,
          2.42609371, 0.73400685]],

        [[0.80830096, 0.32199605, 0.99817655, ..., 0.31176543,
          0.54992302, 0.65464931],
         [0.78325561, 0.57114772, 0.39145505, ..., 0.85120686,
          0.37944036, 0.17407165],
         [0.30716032, 0.34574524, 0.63277452, ..., 0.67189223,
          0.11769581, 0.6188562 ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]])
Dimensions without coordinates: rep, rec, y, x

In [13]:
# cov
cx = xcentral.xStatsAccumCov.from_vals(xx, yy, axis='rec', broadcast=True, moments=(2,2))

c = central.StatsAccumCov.from_vals(x, y, broadcast=True, moments=(2,2))

/Users/wpk/.conda/envs/py37/lib/python3.7/site-packages/numba/core/dispatcher.py:238: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ModuleNotFoundError(No module named 'numba.targets')'.
  entrypoints.init_all()


In [25]:
xcentral.xStatsAccum.from_resample_vals(x, w=w, nrep=20)

In [17]:
central.StatsAccum.from_resample_vals(x=x, w=w, nrep=50, resample_kws=dict(parallel=True))

In [664]:

50_000 * (2 * .65 - 1)

15000.000000000002

In [650]:
sx._dims_mom

('mom_0',)

In [651]:
sx.rename({'mom_0':'moment'})._dims_mom

('moment',)

In [633]:
sx._wrap_xarray_method('rename', new_name_or_name_dict={'mom_0': 'moment'})

In [ ]:
sx.rename()

In [628]:
sx.new_like(getattr(sx._xdata, 'rename')({'mom_0':'moment'}))

In [624]:
sx.rename({'mom_0':'moment'})

NameError: name 'new_name_ord_dict' is not defined

In [617]:
sx.assign_coords(x=np.arange(20)).values

<xarray.DataArray (x: 20, y: 3, mom_0: 3)>
array([[[47.0995782 ,  0.43927201,  0.08204166],
        [47.11065282,  0.47151242,  0.08700602],
        [52.29310235,  0.4763478 ,  0.09280659]],

       [[53.45052988,  0.48181519,  0.0940888 ],
        [49.45413799,  0.48222069,  0.07309364],
        [49.24814741,  0.47780764,  0.08680222]],

       [[50.27019802,  0.50745846,  0.07587962],
        [46.62234422,  0.525389  ,  0.08118025],
        [48.98022238,  0.50209776,  0.08471788]],

       [[45.30727946,  0.50358085,  0.0780454 ],
        [46.66313708,  0.47752805,  0.0724916 ],
        [47.51326218,  0.42152469,  0.08666731]],

       [[48.4348835 ,  0.50693574,  0.09420728],
        [49.6966214 ,  0.47571646,  0.09194322],
        [55.0935161 ,  0.46449306,  0.06256905]],

...

       [[47.71268952,  0.47603831,  0.08869626],
        [50.78534121,  0.49510331,  0.07890716],
        [48.17209276,  0.51903405,  0.07459934]],

       [[51.98284372,  0.50504937,  0.08389553],
        [48.66703221,  0.43979494,  0.08390941],
        [55.99522845,  0.48052369,  0.08152752]],

       [[50.16927259,  0.50154622,  0.08300775],
        [55.56092804,  0.51857632,  0.08473503],
        [55.15680735,  0.45095617,  0.09986471]],

       [[49.35661499,  0.53218125,  0.07572015],
        [51.97241888,  0.53569056,  0.0728118 ],
        [52.2629804 ,  0.52638474,  0.10305747]],

       [[47.60886715,  0.47138079,  0.08851471],
        [45.71575558,  0.4157857 ,  0.08043452],
        [47.07341821,  0.50042472,  0.07396538]]])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Dimensions without coordinates: y, mom_0

In [612]:
sx.block(3, 'y').values

<xarray.DataArray (y: 1, x: 20, mom_0: 3)>
array([[[1.46503333e+02, 4.62873349e-01, 8.77483682e-02],
        [1.52152815e+02, 4.80649844e-01, 8.49101633e-02],
        [1.45872765e+02, 5.11389251e-01, 8.06383608e-02],
        [1.39483679e+02, 4.66913756e-01, 8.03005348e-02],
        [1.53225021e+02, 4.81549485e-01, 8.24164884e-02],
        [1.53384901e+02, 5.19780481e-01, 8.04826113e-02],
        [1.58405294e+02, 4.97239905e-01, 7.62678940e-02],
        [1.51150961e+02, 4.99249828e-01, 8.07914540e-02],
        [1.46099615e+02, 5.10040716e-01, 8.76600092e-02],
        [1.53427744e+02, 4.72483792e-01, 7.47813665e-02],
        [1.47589059e+02, 5.04283321e-01, 9.17288084e-02],
        [1.48272788e+02, 4.93980507e-01, 8.05730216e-02],
        [1.54882232e+02, 4.93362913e-01, 8.48637928e-02],
        [1.50904991e+02, 4.82291392e-01, 7.98293282e-02],
        [1.52892049e+02, 5.20143097e-01, 7.44857747e-02],
        [1.46670123e+02, 4.96761121e-01, 8.09803412e-02],
        [1.56645104e+02, 4.76008818e-01, 8.37479590e-02],
        [1.60887008e+02, 4.90083657e-01, 9.02295065e-02],
        [1.53592014e+02, 5.31396344e-01, 8.40531161e-02],
        [1.40398041e+02, 4.63016186e-01, 8.22247789e-02]]])
Dimensions without coordinates: y, x, mom_0

In [601]:
%%timeit -n 1 -r 1
sx.block2(5)

359 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [603]:
%%timeit -n 1 -r 1
sx.block1(5)

3.96 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [600]:
sx.block2(5).data - s.block(5)

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [588]:
sx.block(2, dims=['x','y','mom_0']).values

<xarray.DataArray (x: 10, y: 3, mom_0: 3)>
array([[[1.00550108e+02, 4.61887158e-01, 8.88963706e-02],
        [9.65647908e+01, 4.76996492e-01, 7.99096624e-02],
        [1.01541250e+02, 4.77055832e-01, 8.98949639e-02]],

       [[9.55774775e+01, 5.05620332e-01, 7.69100322e-02],
        [9.32854813e+01, 5.01448059e-01, 7.74066944e-02],
        [9.64934846e+01, 4.62423685e-01, 8.73004029e-02]],

       [[9.79665343e+01, 5.27244360e-01, 9.10567540e-02],
        [1.03045446e+02, 4.73796271e-01, 8.41012313e-02],
        [1.05597941e+02, 5.02254645e-01, 6.96461493e-02]],

       [[1.03555485e+02, 5.11903283e-01, 7.72137670e-02],
        [1.06031759e+02, 5.07448163e-01, 7.56205805e-02],
        [9.99690107e+01, 4.74262075e-01, 8.19590521e-02]],

       [[1.01809124e+02, 5.00109924e-01, 7.86041240e-02],
        [9.49094980e+01, 5.18608122e-01, 8.36235763e-02],
        [1.02808736e+02, 4.55917330e-01, 8.01448020e-02]],

       [[9.59575190e+01, 4.88697070e-01, 8.19292544e-02],
        [1.00084027e+02, 4.94037323e-01, 8.87655220e-02],
        [9.98203013e+01, 5.14235722e-01, 8.72693206e-02]],

       [[1.00003489e+02, 5.45717950e-01, 8.36504359e-02],
        [1.01556361e+02, 4.65076954e-01, 7.99913001e-02],
        [1.04227374e+02, 4.54660844e-01, 7.87565499e-02]],

       [[9.63962613e+01, 5.09836340e-01, 8.08282918e-02],
        [9.99669233e+01, 4.80598073e-01, 7.21670384e-02],
        [1.03198987e+02, 5.34845664e-01, 7.89845645e-02]],

       [[1.02152116e+02, 5.03328895e-01, 8.34625895e-02],
        [1.04227960e+02, 4.81791026e-01, 8.58943631e-02],
        [1.11152036e+02, 4.65851444e-01, 9.08455071e-02]],

       [[9.69654821e+01, 5.02328969e-01, 8.29259999e-02],
        [9.76881745e+01, 4.79577925e-01, 7.99585995e-02],
        [9.93363986e+01, 5.14082838e-01, 8.94393638e-02]]])
Dimensions without coordinates: x, y, mom_0

In [585]:
xcentral.xStatsAccum.from_datas(sx.data.reshape(10, 2, 3, 3), axis=1, dims=['x','y','mom_0']).values

<xarray.DataArray (x: 10, y: 3, mom_0: 3)>
array([[[1.00550108e+02, 4.61887158e-01, 8.88963706e-02],
        [9.65647908e+01, 4.76996492e-01, 7.99096624e-02],
        [1.01541250e+02, 4.77055832e-01, 8.98949639e-02]],

       [[9.55774775e+01, 5.05620332e-01, 7.69100322e-02],
        [9.32854813e+01, 5.01448059e-01, 7.74066944e-02],
        [9.64934846e+01, 4.62423685e-01, 8.73004029e-02]],

       [[9.79665343e+01, 5.27244360e-01, 9.10567540e-02],
        [1.03045446e+02, 4.73796271e-01, 8.41012313e-02],
        [1.05597941e+02, 5.02254645e-01, 6.96461493e-02]],

       [[1.03555485e+02, 5.11903283e-01, 7.72137670e-02],
        [1.06031759e+02, 5.07448163e-01, 7.56205805e-02],
        [9.99690107e+01, 4.74262075e-01, 8.19590521e-02]],

       [[1.01809124e+02, 5.00109924e-01, 7.86041240e-02],
        [9.49094980e+01, 5.18608122e-01, 8.36235763e-02],
        [1.02808736e+02, 4.55917330e-01, 8.01448020e-02]],

       [[9.59575190e+01, 4.88697070e-01, 8.19292544e-02],
        [1.00084027e+02, 4.94037323e-01, 8.87655220e-02],
        [9.98203013e+01, 5.14235722e-01, 8.72693206e-02]],

       [[1.00003489e+02, 5.45717950e-01, 8.36504359e-02],
        [1.01556361e+02, 4.65076954e-01, 7.99913001e-02],
        [1.04227374e+02, 4.54660844e-01, 7.87565499e-02]],

       [[9.63962613e+01, 5.09836340e-01, 8.08282918e-02],
        [9.99669233e+01, 4.80598073e-01, 7.21670384e-02],
        [1.03198987e+02, 5.34845664e-01, 7.89845645e-02]],

       [[1.02152116e+02, 5.03328895e-01, 8.34625895e-02],
        [1.04227960e+02, 4.81791026e-01, 8.58943631e-02],
        [1.11152036e+02, 4.65851444e-01, 9.08455071e-02]],

       [[9.69654821e+01, 5.02328969e-01, 8.29259999e-02],
        [9.76881745e+01, 4.79577925e-01, 7.99585995e-02],
        [9.93363986e+01, 5.14082838e-01, 8.94393638e-02]]])
Dimensions without coordinates: x, y, mom_0

In [583]:
sx.values

<xarray.DataArray (x: 20, y: 3, mom_0: 3)>
array([[[47.0995782 ,  0.43927201,  0.08204166],
        [47.11065282,  0.47151242,  0.08700602],
        [52.29310235,  0.4763478 ,  0.09280659]],

       [[53.45052988,  0.48181519,  0.0940888 ],
        [49.45413799,  0.48222069,  0.07309364],
        [49.24814741,  0.47780764,  0.08680222]],

       [[50.27019802,  0.50745846,  0.07587962],
        [46.62234422,  0.525389  ,  0.08118025],
        [48.98022238,  0.50209776,  0.08471788]],

       [[45.30727946,  0.50358085,  0.0780454 ],
        [46.66313708,  0.47752805,  0.0724916 ],
        [47.51326218,  0.42152469,  0.08666731]],

       [[48.4348835 ,  0.50693574,  0.09420728],
        [49.6966214 ,  0.47571646,  0.09194322],
        [55.0935161 ,  0.46449306,  0.06256905]],

...

       [[47.71268952,  0.47603831,  0.08869626],
        [50.78534121,  0.49510331,  0.07890716],
        [48.17209276,  0.51903405,  0.07459934]],

       [[51.98284372,  0.50504937,  0.08389553],
        [48.66703221,  0.43979494,  0.08390941],
        [55.99522845,  0.48052369,  0.08152752]],

       [[50.16927259,  0.50154622,  0.08300775],
        [55.56092804,  0.51857632,  0.08473503],
        [55.15680735,  0.45095617,  0.09986471]],

       [[49.35661499,  0.53218125,  0.07572015],
        [51.97241888,  0.53569056,  0.0728118 ],
        [52.2629804 ,  0.52638474,  0.10305747]],

       [[47.60886715,  0.47138079,  0.08851471],
        [45.71575558,  0.4157857 ,  0.08043452],
        [47.07341821,  0.50042472,  0.07396538]]])
Dimensions without coordinates: x, y, mom_0

In [582]:
sx.block2(2, axis='x').values

<xarray.DataArray (y: 10, mom_0: 3)>
array([[[1.00550108e+02, 4.61887158e-01, 8.88963706e-02],
        [9.65647908e+01, 4.76996492e-01, 7.99096624e-02],
        [1.01541250e+02, 4.77055832e-01, 8.98949639e-02]],

       [[9.55774775e+01, 5.05620332e-01, 7.69100322e-02],
        [9.32854813e+01, 5.01448059e-01, 7.74066944e-02],
        [9.64934846e+01, 4.62423685e-01, 8.73004029e-02]],

       [[9.79665343e+01, 5.27244360e-01, 9.10567540e-02],
        [1.03045446e+02, 4.73796271e-01, 8.41012313e-02],
        [1.05597941e+02, 5.02254645e-01, 6.96461493e-02]],

       [[1.03555485e+02, 5.11903283e-01, 7.72137670e-02],
        [1.06031759e+02, 5.07448163e-01, 7.56205805e-02],
        [9.99690107e+01, 4.74262075e-01, 8.19590521e-02]],

       [[1.01809124e+02, 5.00109924e-01, 7.86041240e-02],
        [9.49094980e+01, 5.18608122e-01, 8.36235763e-02],
        [1.02808736e+02, 4.55917330e-01, 8.01448020e-02]],

       [[9.59575190e+01, 4.88697070e-01, 8.19292544e-02],
        [1.00084027e+02, 4.94037323e-01, 8.87655220e-02],
        [9.98203013e+01, 5.14235722e-01, 8.72693206e-02]],

       [[1.00003489e+02, 5.45717950e-01, 8.36504359e-02],
        [1.01556361e+02, 4.65076954e-01, 7.99913001e-02],
        [1.04227374e+02, 4.54660844e-01, 7.87565499e-02]],

       [[9.63962613e+01, 5.09836340e-01, 8.08282918e-02],
        [9.99669233e+01, 4.80598073e-01, 7.21670384e-02],
        [1.03198987e+02, 5.34845664e-01, 7.89845645e-02]],

       [[1.02152116e+02, 5.03328895e-01, 8.34625895e-02],
        [1.04227960e+02, 4.81791026e-01, 8.58943631e-02],
        [1.11152036e+02, 4.65851444e-01, 9.08455071e-02]],

       [[9.69654821e+01, 5.02328969e-01, 8.29259999e-02],
        [9.76881745e+01, 4.79577925e-01, 7.99585995e-02],
        [9.93363986e+01, 5.14082838e-01, 8.94393638e-02]]])
Dimensions without coordinates: y, mom_0, mom_0

In [526]:
xcentral.xStatsAccum.from_datas(sx.data, axis=1, dims=['y','mom_0']).values

<xarray.DataArray (y: 20, mom_0: 3)>
array([[1.46503333e+02, 4.62873349e-01, 8.77483682e-02],
       [1.52152815e+02, 4.80649844e-01, 8.49101633e-02],
       [1.45872765e+02, 5.11389251e-01, 8.06383608e-02],
       [1.39483679e+02, 4.66913756e-01, 8.03005348e-02],
       [1.53225021e+02, 4.81549485e-01, 8.24164884e-02],
       [1.53384901e+02, 5.19780481e-01, 8.04826113e-02],
       [1.58405294e+02, 4.97239905e-01, 7.62678940e-02],
       [1.51150961e+02, 4.99249828e-01, 8.07914540e-02],
       [1.46099615e+02, 5.10040716e-01, 8.76600092e-02],
       [1.53427744e+02, 4.72483792e-01, 7.47813665e-02],
       [1.47589059e+02, 5.04283321e-01, 9.17288084e-02],
       [1.48272788e+02, 4.93980507e-01, 8.05730216e-02],
       [1.54882232e+02, 4.93362913e-01, 8.48637928e-02],
       [1.50904991e+02, 4.82291392e-01, 7.98293282e-02],
       [1.52892049e+02, 5.20143097e-01, 7.44857747e-02],
       [1.46670123e+02, 4.96761121e-01, 8.09803412e-02],
       [1.56645104e+02, 4.76008818e-01, 8.37479590e-02],
       [1.60887008e+02, 4.90083657e-01, 9.02295065e-02],
       [1.53592014e+02, 5.31396344e-01, 8.40531161e-02],
       [1.40398041e+02, 4.63016186e-01, 8.22247789e-02]])
Dimensions without coordinates: y, mom_0

In [521]:
s.block(2)

In [522]:
sx.block(2)

In [523]:
sx.block2(2)

UnboundLocalError: local variable 'dims_total' referenced before assignment

In [497]:
%%timeit -n 1 -r 1
s.block(2)

143 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [512]:
xcentral.xStatsAccum.from_datas(sx.values, axis=1).values

<xarray.DataArray (x: 20, mom_0: 3)>
array([[1.46503333e+02, 4.62873349e-01, 8.77483682e-02],
       [1.52152815e+02, 4.80649844e-01, 8.49101633e-02],
       [1.45872765e+02, 5.11389251e-01, 8.06383608e-02],
       [1.39483679e+02, 4.66913756e-01, 8.03005348e-02],
       [1.53225021e+02, 4.81549485e-01, 8.24164884e-02],
       [1.53384901e+02, 5.19780481e-01, 8.04826113e-02],
       [1.58405294e+02, 4.97239905e-01, 7.62678940e-02],
       [1.51150961e+02, 4.99249828e-01, 8.07914540e-02],
       [1.46099615e+02, 5.10040716e-01, 8.76600092e-02],
       [1.53427744e+02, 4.72483792e-01, 7.47813665e-02],
       [1.47589059e+02, 5.04283321e-01, 9.17288084e-02],
       [1.48272788e+02, 4.93980507e-01, 8.05730216e-02],
       [1.54882232e+02, 4.93362913e-01, 8.48637928e-02],
       [1.50904991e+02, 4.82291392e-01, 7.98293282e-02],
       [1.52892049e+02, 5.20143097e-01, 7.44857747e-02],
       [1.46670123e+02, 4.96761121e-01, 8.09803412e-02],
       [1.56645104e+02, 4.76008818e-01, 8.37479590e-02],
       [1.60887008e+02, 4.90083657e-01, 9.02295065e-02],
       [1.53592014e+02, 5.31396344e-01, 8.40531161e-02],
       [1.40398041e+02, 4.63016186e-01, 8.22247789e-02]])
Dimensions without coordinates: x, mom_0

In [509]:
sx.values

<xarray.DataArray (x: 20, y: 3, mom_0: 3)>
array([[[47.0995782 ,  0.43927201,  0.08204166],
        [47.11065282,  0.47151242,  0.08700602],
        [52.29310235,  0.4763478 ,  0.09280659]],

       [[53.45052988,  0.48181519,  0.0940888 ],
        [49.45413799,  0.48222069,  0.07309364],
        [49.24814741,  0.47780764,  0.08680222]],

       [[50.27019802,  0.50745846,  0.07587962],
        [46.62234422,  0.525389  ,  0.08118025],
        [48.98022238,  0.50209776,  0.08471788]],

       [[45.30727946,  0.50358085,  0.0780454 ],
        [46.66313708,  0.47752805,  0.0724916 ],
        [47.51326218,  0.42152469,  0.08666731]],

       [[48.4348835 ,  0.50693574,  0.09420728],
        [49.6966214 ,  0.47571646,  0.09194322],
        [55.0935161 ,  0.46449306,  0.06256905]],

...

       [[47.71268952,  0.47603831,  0.08869626],
        [50.78534121,  0.49510331,  0.07890716],
        [48.17209276,  0.51903405,  0.07459934]],

       [[51.98284372,  0.50504937,  0.08389553],
        [48.66703221,  0.43979494,  0.08390941],
        [55.99522845,  0.48052369,  0.08152752]],

       [[50.16927259,  0.50154622,  0.08300775],
        [55.56092804,  0.51857632,  0.08473503],
        [55.15680735,  0.45095617,  0.09986471]],

       [[49.35661499,  0.53218125,  0.07572015],
        [51.97241888,  0.53569056,  0.0728118 ],
        [52.2629804 ,  0.52638474,  0.10305747]],

       [[47.60886715,  0.47138079,  0.08851471],
        [45.71575558,  0.4157857 ,  0.08043452],
        [47.07341821,  0.50042472,  0.07396538]]])
Dimensions without coordinates: x, y, mom_0

In [506]:
sx.assign_coords(x=np.arange(20)).isel(x=0, drop=True).coords

Coordinates:
    *empty*

In [501]:
sx.dims

('x', 'y', 'mom_0')

In [500]:
%%timeit -n 1 -r 1
sx.block(3).data 

4.14 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
np.tile()

In [355]:
np.repeat?

In [347]:
da = sx.values

In [353]:
da.sizes['x']

20

In [388]:
np.repeat(np.arange(10), 2)

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9])

In [391]:
np.tile(np.arange(2), 10)

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [390]:
np.repeat(np.arange(2), 10)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [407]:
new = (
    da
    .rename({'x': 'z'})
    .assign_coords(
        x=('z', np.repeat(np.arange(10), 2)),
        b=('z', np.tile(np.arange(2), 10))
    )
    .set_index({'z': ('x','b')})
    .unstack('z')
)
    

In [439]:
sx._wrap_axis('y')

1

In [440]:
sx.dims

('x', 'y', 'mom_0')

In [430]:
def blockdata(da, block_size, dim='x'):
    
    n = da.sizes[dim]
    
    if block_size is None:
        block_size = n
        nblock = 1
    else:
        nblock = n // block_size
        
    da = da.isel(**{dim: slice(None, nblock * block_size, None)})
    
    z = '_tmp_{}'.format(dim)
    b = '{}_block'.format(dim)
    new = (
        da
        .isel(**{dim : slice(None, nblock * block_size, None)})
        .rename({dim : z})
        .assign_coords(
            **{
                dim: (z, np.repeat(np.arange(nblock), block_size)),
                b:   (z, np.tile(np.arange(block_size), nblock))
            }
        )
        .set_index({z: (dim, b)})
        .unstack(z)
        .transpose(*(b,) + da.dims)
    )
    
    del new[dim]
    del new[b]
    
    return new



TypeError: _wrap_like() missing 1 required positional argument: 'x'

In [431]:
blockdata(da, 3)#.transpose(*('x_block',) + da.dims)

<xarray.DataArray (x_block: 3, x: 6, y: 3, mom_0: 3)>
array([[[[53.9806198 ,  0.46621331,  0.0838397 ],
         [42.40435898,  0.46999624,  0.07906751],
         [48.53806702,  0.56492376,  0.06264107]],

        [[46.84837342,  0.55010598,  0.08909043],
         [55.75337241,  0.52193368,  0.09270573],
         [48.52249731,  0.54612623,  0.07342019]],

        [[47.46018686,  0.52312373,  0.09862233],
         [47.95130618,  0.48760533,  0.08857919],
         [48.787946  ,  0.47193481,  0.08521729]],

        [[53.07424665,  0.41240595,  0.08357522],
         [53.017431  ,  0.50370342,  0.09048387],
         [48.69547874,  0.53674664,  0.08483372]],

        [[49.38510354,  0.48860625,  0.07660554],
         [51.4424247 ,  0.52239686,  0.08719331],
         [47.45515212,  0.50832387,  0.09579028]],

...

        [[49.22988358,  0.5028811 ,  0.06553325],
         [51.55057081,  0.54009863,  0.08899201],
         [53.5889934 ,  0.4857182 ,  0.0828143 ]],

        [[49.18599232,  0.53691087,  0.07534248],
         [48.70337253,  0.45481243,  0.08137814],
         [48.94628753,  0.51855245,  0.0864078 ]],

        [[49.87316925,  0.45696814,  0.07155436],
         [48.73357522,  0.50117532,  0.0997453 ],
         [49.94737516,  0.46265478,  0.08228995]],

        [[49.67437784,  0.54100429,  0.08775518],
         [46.24650064,  0.46476972,  0.0750003 ],
         [54.82737566,  0.55267355,  0.08235927]],

        [[48.96942268,  0.42083257,  0.07206718],
         [50.12303535,  0.53004091,  0.08630755],
         [54.9110624 ,  0.54281877,  0.08484089]]]])
Dimensions without coordinates: x_block, x, y, mom_0

In [341]:
s.reshape((10, 2, 3)).reduce(1).values - s.block(2).values

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [339]:
s.block(2).values

array([[[1.02530470e+02, 4.62917455e-01, 8.19537484e-02],
        [9.19950069e+01, 4.93374982e-01, 7.94074182e-02],
        [9.80761873e+01, 5.32528359e-01, 7.00299663e-02]],

       [[9.53949545e+01, 5.07200871e-01, 8.83267532e-02],
        [1.05747293e+02, 5.09491493e-01, 8.63103903e-02],
        [1.00774581e+02, 4.95876533e-01, 7.88063101e-02]],

       [[9.75038872e+01, 5.13202509e-01, 7.29889088e-02],
        [9.73813693e+01, 4.99340721e-01, 8.79080886e-02],
        [9.90037484e+01, 4.71034686e-01, 7.84804554e-02]],

       [[1.00174017e+02, 5.37641337e-01, 8.98856013e-02],
        [9.63169725e+01, 5.05956778e-01, 8.45067707e-02],
        [9.96786694e+01, 4.95334667e-01, 8.42896526e-02]],

       [[1.02260239e+02, 4.72291381e-01, 8.34851354e-02],
        [1.01720804e+02, 4.80294674e-01, 8.67206045e-02],
        [9.76417663e+01, 5.27626177e-01, 8.57055417e-02]],

       [[1.00074399e+02, 4.83104611e-01, 7.82389334e-02],
        [1.03917848e+02, 5.24213176e-01, 9.41006560e-02],
    

In [333]:
s.block(2)

(10, 3)

In [297]:


3 *(s.shape[0] // 3)

18

In [ ]:
sx.reshape()

In [312]:
(
    sx.values.rename({'x': 'z'})
    
)

<xarray.DataArray (z: 20, y: 3, mom_0: 3)>
array([[[51.04581411,  0.48191072,  0.0918383 ],
        [52.35375073,  0.5456695 ,  0.08527619],
        [47.96425863,  0.5246168 ,  0.08062568]],

       [[49.75848097,  0.49493491,  0.09071702],
        [56.97285892,  0.57277268,  0.070476  ],
        [53.45684869,  0.49067074,  0.08609262]],

       [[50.35673157,  0.53479047,  0.0960344 ],
        [51.26892438,  0.51514723,  0.08177126],
        [50.03596947,  0.54074888,  0.06527812]],

       [[46.57973495,  0.47538216,  0.09258512],
        [53.35440472,  0.45849722,  0.08731797],
        [50.31730154,  0.47575053,  0.08070734]],

       [[51.59996079,  0.49164295,  0.08437141],
        [49.48515545,  0.50933501,  0.07871171],
        [53.15201998,  0.49494463,  0.07749728]],

...

       [[47.78215349,  0.5009523 ,  0.07534697],
        [53.18514876,  0.52008731,  0.09903011],
        [52.15840927,  0.48140341,  0.0766887 ]],

       [[47.8585308 ,  0.48568704,  0.09256512],
        [50.48358954,  0.52102847,  0.08822814],
        [44.35293231,  0.48096115,  0.0740663 ]],

       [[52.4364086 ,  0.47577063,  0.08566791],
        [47.43504268,  0.45501272,  0.0732211 ],
        [49.42650804,  0.51409793,  0.07755212]],

       [[49.11596629,  0.48894223,  0.08757784],
        [54.80542333,  0.45619992,  0.09847625],
        [51.62990978,  0.46205244,  0.09196834]],

       [[52.52206674,  0.41921699,  0.07217399],
        [50.44011827,  0.3974465 ,  0.064503  ],
        [51.97924146,  0.5713903 ,  0.06604444]]])
Dimensions without coordinates: z, y, mom_0

In [305]:
v = np.arange(20)

In [308]:
v.reshape(2, 10)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [ ]:
v[]

In [289]:
s.data.shape[]

(20, 3, 3)

In [287]:
s.reshape((2, 10, 3)).reduce(1)

In [280]:
s.reshape()

In [155]:
freq = xr.DataArray(np.random.rand(2,100), dims=['rep','rec'], coords={'rep':range(2)})
freq.coords

Coordinates:
  * rep      (rep) int64 0 1

In [156]:
xcentral.xStatsAccum.from_data(s.data, dims=sx.dims).dims

('x', 'y', 'mom_0')

In [157]:
idx = np.random.choice(20, size=(10, 20), replace=True)

In [158]:
xidx = xr.DataArray(idx, dims=['rep','x'])

In [159]:
r = s.resample(idx)
rx = sx.sel(x=xidx)

In [162]:
np.testing.assert_allclose(r.reduce(1).values, rx.reduce('x').values)

In [163]:
#%%timeit -n 1 -r 1
freq = central.randsamp_freq(indices=idx)

In [164]:
np.testing.assert_allclose(r.reduce(1).values, s.resample_and_reduce(freq).values)

In [165]:
np.testing.assert_allclose(r.reduce(1).values, s.resample_and_reduce(indices=idx).values)

In [168]:
np.testing.assert_allclose(rx.reduce('x').values, r.reduce(1))

In [169]:
np.testing.assert_allclose(s.resample_and_reduce(freq).values, sx.resample_and_reduce(freq).values)

In [171]:
np.testing.assert_allclose(s.resample_and_reduce(freq).values, sx.resample_and_reduce(indices=idx).values)

In [74]:
sx.dims

('x', 'y', 'mom_0')

In [73]:
sx.resample_and_reduce(freq, dims=['rep','y']).values

<xarray.DataArray (rep: 10, y: 3, mom_0: 3)>
array([[[9.95034688e+02, 5.06400978e-01, 8.52334130e-02],
        [9.69972008e+02, 5.10975056e-01, 8.13645389e-02],
        [9.93544784e+02, 4.94057649e-01, 8.49261695e-02]],

       [[9.95146922e+02, 5.04989576e-01, 8.25647428e-02],
        [9.65580336e+02, 5.05786246e-01, 8.11629610e-02],
        [1.00026336e+03, 5.00487697e-01, 8.37466568e-02]],

       [[1.01338369e+03, 5.06482564e-01, 8.32851177e-02],
        [9.80957470e+02, 5.03771610e-01, 7.87985435e-02],
        [9.89359651e+02, 4.88132306e-01, 8.58563155e-02]],

       [[9.98333344e+02, 5.01537253e-01, 8.71290865e-02],
        [9.97517262e+02, 4.99439996e-01, 8.19081459e-02],
        [9.89768711e+02, 5.05927293e-01, 8.50634557e-02]],

       [[1.00080379e+03, 5.03734208e-01, 8.56502150e-02],
        [9.65756516e+02, 5.11193752e-01, 8.17921291e-02],
        [9.94930192e+02, 4.92761707e-01, 8.74237153e-02]],

       [[9.91410772e+02, 5.11666952e-01, 8.14193556e-02],
        [9.71982025e+02, 5.16556234e-01, 8.57242307e-02],
        [9.73440928e+02, 4.96377211e-01, 8.50796758e-02]],

       [[9.96055548e+02, 5.16374929e-01, 8.02856400e-02],
        [9.76339059e+02, 4.97585590e-01, 8.28939806e-02],
        [9.96165314e+02, 4.94262142e-01, 8.32523782e-02]],

       [[1.00447858e+03, 5.06780686e-01, 8.03660717e-02],
        [9.83206297e+02, 4.97867609e-01, 8.52314633e-02],
        [9.91721833e+02, 4.99103593e-01, 8.07070194e-02]],

       [[1.00725424e+03, 4.92304642e-01, 8.41324188e-02],
        [9.85635926e+02, 5.15878315e-01, 8.28856349e-02],
        [1.00288523e+03, 5.09092169e-01, 8.50688736e-02]],

       [[9.99351807e+02, 4.92780751e-01, 8.64433255e-02],
        [9.79575789e+02, 5.06207370e-01, 8.15702139e-02],
        [9.83130058e+02, 4.98807373e-01, 8.16797708e-02]]])
Dimensions without coordinates: rep, y, mom_0

In [61]:
sx.resample_and_reduce(freq).values.values

<xarray.DataArray (dim_0: 10, dim_1: 3, mom_0: 3)>
array([[[9.95034688e+02, 5.06400978e-01, 8.52334130e-02],
        [9.69972008e+02, 5.10975056e-01, 8.13645389e-02],
        [9.93544784e+02, 4.94057649e-01, 8.49261695e-02]],

       [[9.95146922e+02, 5.04989576e-01, 8.25647428e-02],
        [9.65580336e+02, 5.05786246e-01, 8.11629610e-02],
        [1.00026336e+03, 5.00487697e-01, 8.37466568e-02]],

       [[1.01338369e+03, 5.06482564e-01, 8.32851177e-02],
        [9.80957470e+02, 5.03771610e-01, 7.87985435e-02],
        [9.89359651e+02, 4.88132306e-01, 8.58563155e-02]],

       [[9.98333344e+02, 5.01537253e-01, 8.71290865e-02],
        [9.97517262e+02, 4.99439996e-01, 8.19081459e-02],
        [9.89768711e+02, 5.05927293e-01, 8.50634557e-02]],

       [[1.00080379e+03, 5.03734208e-01, 8.56502150e-02],
        [9.65756516e+02, 5.11193752e-01, 8.17921291e-02],
        [9.94930192e+02, 4.92761707e-01, 8.74237153e-02]],

       [[9.91410772e+02, 5.11666952e-01, 8.14193556e-02],
        [9.71982025e+02, 5.16556234e-01, 8.57242307e-02],
        [9.73440928e+02, 4.96377211e-01, 8.50796758e-02]],

       [[9.96055548e+02, 5.16374929e-01, 8.02856400e-02],
        [9.76339059e+02, 4.97585590e-01, 8.28939806e-02],
        [9.96165314e+02, 4.94262142e-01, 8.32523782e-02]],

       [[1.00447858e+03, 5.06780686e-01, 8.03660717e-02],
        [9.83206297e+02, 4.97867609e-01, 8.52314633e-02],
        [9.91721833e+02, 4.99103593e-01, 8.07070194e-02]],

       [[1.00725424e+03, 4.92304642e-01, 8.41324188e-02],
        [9.85635926e+02, 5.15878315e-01, 8.28856349e-02],
        [1.00288523e+03, 5.09092169e-01, 8.50688736e-02]],

       [[9.99351807e+02, 4.92780751e-01, 8.64433255e-02],
        [9.79575789e+02, 5.06207370e-01, 8.15702139e-02],
        [9.83130058e+02, 4.98807373e-01, 8.16797708e-02]]])
Dimensions without coordinates: dim_0, dim_1, mom_0

In [56]:
#%%timeit -n 1 -r 1
s.resample_and_reduce(freq)

In [137]:
%%timeit -n 1 -r 1
s.resample(idx).reduce(1)


226 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [106]:
xidx = xr.DataArray(idx, dims=['rep','x'])

In [107]:
rx.reduce(1)

In [108]:
%%timeit -n 1 -r 1
rx.reduce(1)

374 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [682]:
xx 

<xarray.DataArray (x: 2, rec: 100, y: 3)>
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0

In [675]:
zz = xx.astype(np.float, copy=False) 

In [677]:
zz[...] = 0.0

In [678]:
xx

<xarray.DataArray (x: 2, rec: 100, y: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0

In [669]:
x.dims

AttributeError: 'numpy.ndarray' object has no attribute 'dims'

In [668]:
xx.sel(rec=xr.DataArray(idx, dims=('rep','rec')))

<xarray.DataArray (x: 2, rep: 20, rec: 100, y: 3)>
array([[[[0.11413378, 0.96740882, 0.41547863],
         [0.26480503, 0.64551586, 0.83666979],
         [0.20968679, 0.81835694, 0.58638014],
         ...,
         [0.92709572, 0.88810565, 0.62661948],
         [0.0143984 , 0.37211736, 0.5291583 ],
         [0.96919181, 0.63598048, 0.0650079 ]],

        [[0.18065019, 0.01221575, 0.39208233],
         [0.02552378, 0.09434842, 0.83095689],
         [0.60070585, 0.22171802, 0.12990324],
         ...,
         [0.71830089, 0.60418753, 0.02689951],
         [0.09791947, 0.64751318, 0.17396924],
         [0.74802504, 0.28464854, 0.23274928]],

        [[0.96919181, 0.63598048, 0.0650079 ],
         [0.09791947, 0.64751318, 0.17396924],
         [0.15883913, 0.07174457, 0.29881027],
         ...,
         [0.94524206, 0.00800307, 0.98816265],
         [0.82586806, 0.02417953, 0.1216264 ],
         [0.6791873 , 0.39550765, 0.75986895]],

        ...,

        [[0.1259188 , 0.56677805, 0.19156828],
         [0.93425163, 0.52984114, 0.80806418],
         [0.99854544, 0.53264262, 0.7281167 ],
         ...,
         [0.1259188 , 0.56677805, 0.19156828],
         [0.26480503, 0.64551586, 0.83666979],
         [0.74802504, 0.28464854, 0.23274928]],

        [[0.49554345, 0.12905553, 0.64095842],
         [0.99995098, 0.07190527, 0.07750033],
         [0.52090944, 0.65897613, 0.65446292],
         ...,
         [0.99854544, 0.53264262, 0.7281167 ],
         [0.21695918, 0.55866626, 0.06121161],
         [0.93425163, 0.52984114, 0.80806418]],

        [[0.14827637, 0.66950937, 0.05475215],
         [0.92709572, 0.88810565, 0.62661948],
         [0.41108614, 0.56927535, 0.41087678],
         ...,
         [0.1259188 , 0.56677805, 0.19156828],
         [0.2096441 , 0.20017367, 0.03130757],
         [0.69856798, 0.30118577, 0.42969644]]],


       [[[0.01902397, 0.88536514, 0.05744012],
         [0.4255346 , 0.27071013, 0.8329724 ],
         [0.5644828 , 0.83290208, 0.10554253],
         ...,
         [0.64964539, 0.45105929, 0.29660239],
         [0.2491791 , 0.44380376, 0.67594692],
         [0.70502842, 0.90203838, 0.0962265 ]],

        [[0.83725428, 0.22778351, 0.68039144],
         [0.32950851, 0.05018998, 0.9214166 ],
         [0.68006681, 0.53689029, 0.51412188],
         ...,
         [0.6008424 , 0.46043834, 0.4752563 ],
         [0.72322887, 0.3622542 , 0.83427519],
         [0.28608991, 0.10558184, 0.819057  ]],

        [[0.70502842, 0.90203838, 0.0962265 ],
         [0.72322887, 0.3622542 , 0.83427519],
         [0.21162909, 0.11311972, 0.52987768],
         ...,
         [0.19962637, 0.62207448, 0.13131591],
         [0.35873596, 0.16434839, 0.5804439 ],
         [0.40170108, 0.02474845, 0.80180007]],

        ...,

        [[0.02215918, 0.46072542, 0.29297362],
         [0.74275875, 0.51255677, 0.40768868],
         [0.72969842, 0.86628541, 0.65152557],
         ...,
         [0.02215918, 0.46072542, 0.29297362],
         [0.4255346 , 0.27071013, 0.8329724 ],
         [0.28608991, 0.10558184, 0.819057  ]],

        [[0.34276443, 0.35229136, 0.42557116],
         [0.27699792, 0.6946457 , 0.7669943 ],
         [0.11208983, 0.98042315, 0.30009574],
         ...,
         [0.72969842, 0.86628541, 0.65152557],
         [0.44806397, 0.16326342, 0.51178606],
         [0.74275875, 0.51255677, 0.40768868]],

        [[0.32749199, 0.34235684, 0.69008414],
         [0.64964539, 0.45105929, 0.29660239],
         [0.3366993 , 0.24058176, 0.25040077],
         ...,
         [0.02215918, 0.46072542, 0.29297362],
         [0.82106587, 0.43923241, 0.85810828],
         [0.76156012, 0.13546131, 0.25428433]]]])
Dimensions without coordinates: x, rep, rec, y

In [640]:
v = sx.values.stack(z=('x','y'))

In [642]:
v[...] = 0.0

In [645]:
sx.values

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: x, y, mom_0

In [627]:
new = sx.assign_coords(x=range(2)).assign_attrs(B=2)

In [632]:
sx.values.transpose('mom_0')

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: x, y, mom_0

In [631]:
sx._wrap_xarray_method('transpose', 'y',...)

ValueError: passed data shape (3, 2, 3) must equal (2, 3, 3)

In [630]:
sx.values

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: x, y, mom_0

In [628]:
new.values

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Coordinates:
  * x        (x) int64 0 1
Dimensions without coordinates: y, mom_0
Attributes:
    B:        2

In [605]:
b = sx.values.assign_coords(x=range(2))

In [607]:
b[...] = 0.0

In [600]:
sx.coords

Coordinates:
  * x        (x) int64 0 1

In [593]:
new = sx.transpose('y','x')

In [594]:
new._data[...] = 0.0

In [573]:
dims = ['a','b',...,'mom_0']
if 'mom_0' in dims:
    dims.pop(dims.index('mom_0'))

In [574]:
dims

['a', 'b', Ellipsis]

In [567]:
sx.transpose('y',...,'x').shape

(3, 2)

In [528]:
new = sx.values.transpose('mom_0','y','x')

In [538]:
xcentral.xStatsAccum.from_data()

('mom_0',)

In [ ]:
xcentral.xStatsAccum.from_data

In [537]:
xcentral.xStatsAccum.from_data(new.transpose(...,*sx._dims_mom))

In [ ]:
xcentral.xStatsAccum.from_datas(new, shape=sx.shape, moments=sx.moments, )

In [525]:
sx._dims_mom

('mom_0',)

In [494]:
xcentral.xStatsAccum.from_datas(sx.values, axis='x').dims

('y', 'mom_0')

In [497]:
sx.reduce(0).dims

('y', 'mom_0')

In [463]:
sx.shape

(20, 3)

In [469]:
sx.reduce('x'

('dim_0', 'mom_0')

In [466]:
sx.reshape((2,)

<xarray.DataArray (dim_0: 2, dim_1: 10, dim_2: 3, mom_0: 3)>
array([[[[52.21531507,  0.52542044,  0.07990223],
         [47.96980534,  0.54136798,  0.08038598],
         [50.04557375,  0.51924632,  0.07538475]],

        [[50.68106984,  0.50948904,  0.07594194],
         [51.47344941,  0.48931379,  0.08163606],
         [48.9459562 ,  0.46554395,  0.07010526]],

        [[52.9469106 ,  0.47677228,  0.0724415 ],
         [49.41219318,  0.49429247,  0.0774639 ],
         [51.36656638,  0.49100141,  0.06942536]],

        [[52.5837636 ,  0.49425151,  0.08757406],
         [50.09807358,  0.49646461,  0.08194543],
         [48.4009103 ,  0.43236883,  0.08563012]],

        [[51.54857502,  0.44659675,  0.09132782],
         [52.36126204,  0.4549441 ,  0.06223141],
         [49.26117015,  0.48230335,  0.07350974]],

        [[52.16083227,  0.43413958,  0.08514121],
         [52.94900422,  0.492003  ,  0.08769643],
         [48.11992721,  0.47720097,  0.08358886]],

        [[50.674906  ,  0.50817051,  0.07921779],
         [47.87442129,  0.56206553,  0.06863487],
         [47.18161662,  0.46899342,  0.08573214]],

        [[54.54771362,  0.46836305,  0.06806273],
         [47.0968063 ,  0.46706302,  0.08853185],
         [47.74560105,  0.44011479,  0.07032594]],

        [[52.38507987,  0.53178384,  0.09017601],
         [50.09488419,  0.55023905,  0.08776668],
         [52.59584164,  0.5147462 ,  0.08443981]],

        [[53.05654137,  0.49315205,  0.0890261 ],
         [52.07142651,  0.49480869,  0.08280832],
         [47.29045634,  0.5536053 ,  0.09755244]]],


       [[[49.93265322,  0.57334313,  0.0830369 ],
         [47.67786931,  0.47207757,  0.0819814 ],
         [48.35696745,  0.49205384,  0.08249119]],

        [[52.34954777,  0.55202262,  0.0784794 ],
         [51.35572936,  0.44271278,  0.07203246],
         [50.72311199,  0.48746153,  0.08925304]],

        [[47.94896698,  0.44589815,  0.08749291],
         [52.3628809 ,  0.53067697,  0.08016774],
         [50.11665624,  0.44441879,  0.09302928]],

        [[48.09227375,  0.47413819,  0.08712038],
         [50.46895461,  0.49438607,  0.07515251],
         [54.33115454,  0.48642235,  0.08196167]],

        [[54.24751967,  0.51675285,  0.09061557],
         [54.30593835,  0.49117329,  0.08242063],
         [50.60961771,  0.52244374,  0.0774554 ]],

        [[50.86632003,  0.50335394,  0.0605746 ],
         [54.98084448,  0.44838151,  0.07266671],
         [55.13579513,  0.48231796,  0.09024816]],

        [[52.9823826 ,  0.47865409,  0.08539312],
         [48.46221924,  0.51301747,  0.0799737 ],
         [49.2273587 ,  0.47736644,  0.07837676]],

        [[48.14059679,  0.45071054,  0.10166265],
         [48.59571235,  0.47032889,  0.08378395],
         [46.76971949,  0.52887984,  0.08238876]],

        [[52.96286152,  0.50861407,  0.08726183],
         [51.12810343,  0.46584042,  0.08431945],
         [49.85789256,  0.48251519,  0.094168  ]],

        [[48.91390939,  0.5021797 ,  0.07452543],
         [46.97161061,  0.51405729,  0.08555413],
         [49.16446826,  0.50356282,  0.07339567]]]])
Dimensions without coordinates: dim_0, dim_1, dim_2, mom_0

In [ ]:
sx = x

In [430]:
s.push_vals(x)

In [437]:
sx.zero()
sx.push_vals(xx, w=2.0, axis='rec')

In [ ]:
sx

In [410]:
sc = central.StatsAccumCov.from_vals(x, y, broadcast=True)

In [411]:
sx = xcentral.xStatsAccumCov.from_vals(x, y, broadcast=True)

In [412]:
sx = xcentral.xStatsAccumCov.from_vals(xx, yy, broadcast=True, axis='rec')

In [413]:
sx2 = xcentral.xStatsAccumCov(sx.moments, shape=sx.shape, data=sx.values)#, moments_dims=['m0','m1'])

In [421]:
ss = sx.stack(z=('x','y'))


In [426]:
ss.values.data#.unstack('z')

array([[[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 4.88273257e-01, -6.85786871e-03, -9.41124914e-04],
        [ 9.39595028e-02, -1.66203944e-03,  7.31775128e-03]],

       [[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 5.03672225e-01, -5.16578683e-03, -1.56130773e-03],
        [ 9.03560809e-02, -2.36783124e-04,  6.50949992e-03]],

       [[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 4.46404624e-01,  7.44873221e-03, -4.13774432e-03],
        [ 9.04832084e-02,  6.30118901e-03,  8.19206178e-03]],

       [[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 4.50895187e-01, -4.22769873e-03, -2.19273862e-05],
        [ 8.33454591e-02,  2.47108455e-04,  6.78485304e-03]],

       [[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 4.84325420e-01,  2.38459382e-03,  1.67847610e-03],
        [ 8.90928868e-02,  2.44511086e-05,  6.93980091e-03]],

       [[ 1.00000000e+02,  4.55869145e-01,  8.24154952e-02],
        [ 4.81

In [382]:
sx2.values.rename(x='x1')

<xarray.DataArray (x1: 2, y: 3, mom_0: 3, mom_1: 3)>
array([[[[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 4.68270229e-01,  8.21107577e-03,  2.69943289e-04],
         [ 8.97222507e-02,  7.62267871e-04,  6.88337917e-03]],

        [[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 4.81187552e-01, -1.25937040e-02,  2.30578034e-03],
         [ 8.40068182e-02, -2.91480525e-03,  8.56536989e-03]],

        [[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 4.92571207e-01,  1.17498715e-02,  2.36643918e-03],
         [ 8.19594064e-02,  1.66744802e-03,  8.00933310e-03]]],


       [[[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 4.89283498e-01, -1.00785345e-03, -4.50563421e-04],
         [ 8.43872944e-02, -4.02413126e-03,  7.89601314e-03]],

        [[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 5.19019205e-01, -1.05315927e-03, -2.68240116e-03],
         [ 9.46097170e-02,  3.14832635e-03,  7.73274135e-03]],

        [[ 1.00000000e+02,  5.26887938e-01,  8.75282713e-02],
         [ 5.19682608e-01,  7.66934784e-03, -6.70872277e-05],
         [ 9.30830560e-02,  4.86355414e-03,  8.67816908e-03]]]])
Dimensions without coordinates: x1, y, mom_0, mom_1

In [371]:
sx2.rename('hello')

ValueError: cannot rename 'new_name_ord_dict' because it is not a variable or dimension in this dataset

In [369]:
sx2.values.rename({'x':'x1'})

<xarray.DataArray (x1: 2, y: 3, mom_0: 3, mom_1: 3)>
array([[[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.18418862e-01,  2.61774216e-03, -2.05030237e-03],
         [ 8.79933966e-02,  1.91288133e-03,  7.84055060e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.69997690e-01,  5.05614150e-03,  1.07061121e-03],
         [ 7.62964636e-02,  5.56989061e-04,  7.12542525e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.50039451e-01, -5.12565252e-04,  2.14899403e-03],
         [ 9.21993706e-02, -1.20464346e-03,  9.70748727e-03]]],


       [[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.85570314e-01, -2.56132035e-03, -1.25263984e-03],
         [ 8.43722227e-02, -1.32782193e-03,  8.02996011e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.45559754e-01, -1.73912077e-03,  4.43022890e-03],
         [ 7.87150494e-02, -4.69588314e-03,  6.34232524e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.00655201e-01,  1.09286039e-02,  5.04243319e-03],
         [ 7.99890824e-02, -7.32948281e-04,  5.47357960e-03]]]])
Coordinates:
  * mom_0    (mom_0) int64 0 1 2
Dimensions without coordinates: x1, y, mom_1
Attributes:
    a:        2

In [368]:
sx2.values.rename('hello')

<xarray.DataArray 'hello' (x: 2, y: 3, mom_0: 3, mom_1: 3)>
array([[[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.18418862e-01,  2.61774216e-03, -2.05030237e-03],
         [ 8.79933966e-02,  1.91288133e-03,  7.84055060e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.69997690e-01,  5.05614150e-03,  1.07061121e-03],
         [ 7.62964636e-02,  5.56989061e-04,  7.12542525e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.50039451e-01, -5.12565252e-04,  2.14899403e-03],
         [ 9.21993706e-02, -1.20464346e-03,  9.70748727e-03]]],


       [[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.85570314e-01, -2.56132035e-03, -1.25263984e-03],
         [ 8.43722227e-02, -1.32782193e-03,  8.02996011e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.45559754e-01, -1.73912077e-03,  4.43022890e-03],
         [ 7.87150494e-02, -4.69588314e-03,  6.34232524e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.00655201e-01,  1.09286039e-02,  5.04243319e-03],
         [ 7.99890824e-02, -7.32948281e-04,  5.47357960e-03]]]])
Coordinates:
  * mom_0    (mom_0) int64 0 1 2
Dimensions without coordinates: x, y, mom_1
Attributes:
    a:        2

In [360]:
sx2.values.assign_attrs(a=1)

<xarray.DataArray (x: 2, y: 3, mom_0: 3, mom_1: 3)>
array([[[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.18418862e-01,  2.61774216e-03, -2.05030237e-03],
         [ 8.79933966e-02,  1.91288133e-03,  7.84055060e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.69997690e-01,  5.05614150e-03,  1.07061121e-03],
         [ 7.62964636e-02,  5.56989061e-04,  7.12542525e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.50039451e-01, -5.12565252e-04,  2.14899403e-03],
         [ 9.21993706e-02, -1.20464346e-03,  9.70748727e-03]]],


       [[[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 4.85570314e-01, -2.56132035e-03, -1.25263984e-03],
         [ 8.43722227e-02, -1.32782193e-03,  8.02996011e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.45559754e-01, -1.73912077e-03,  4.43022890e-03],
         [ 7.87150494e-02, -4.69588314e-03,  6.34232524e-03]],

        [[ 1.00000000e+02,  5.13800166e-01,  8.44624916e-02],
         [ 5.00655201e-01,  1.09286039e-02,  5.04243319e-03],
         [ 7.99890824e-02, -7.32948281e-04,  5.47357960e-03]]]])
Dimensions without coordinates: x, y, mom_0, mom_1
Attributes:
    a:        1

In [ ]:
sx.values.rename()

In [347]:
sx.values.assign_coords()

<xarray.DataArray (x: 2, y: 3, mom_0: 2, mom_1: 2)>
array([[[[7.21519454e-02, 7.17749099e-03],
         [1.00000000e+02, 9.69118335e-02]],

        [[7.90077810e-02, 7.14179069e-03],
         [1.00000000e+02, 9.69118335e-02]],

        [[8.02988902e-02, 8.50869472e-03],
         [1.00000000e+02, 9.69118335e-02]]],


       [[[7.99631159e-02, 6.96682181e-03],
         [1.00000000e+02, 9.69118335e-02]],

        [[8.02569827e-02, 7.84536756e-03],
         [1.00000000e+02, 9.69118335e-02]],

        [[7.73236409e-02, 6.32711165e-03],
         [1.00000000e+02, 9.69118335e-02]]]])
Dimensions without coordinates: x, y, mom_0, mom_1

In [330]:
sc.reduce(0).data - sx.reduce('x')

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [328]:
sx.reduce('x')

In [278]:
s.zero()
s.push_vals(x, w)
s.values

array([[[53.48505531,  0.45581937,  0.07743162],
        [50.20687552,  0.45539711,  0.07584982],
        [48.30067994,  0.50679475,  0.10048263]],

       [[47.56845298,  0.4985016 ,  0.09197212],
        [50.98911977,  0.49509856,  0.09341481],
        [47.45092606,  0.48271688,  0.08039621]]])

In [279]:
sx.zero()
sx.push_vals(x, w)
sx.values

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[53.48505531,  0.45581937,  0.07743162],
        [50.20687552,  0.45539711,  0.07584982],
        [48.30067994,  0.50679475,  0.10048263]],

       [[47.56845298,  0.4985016 ,  0.09197212],
        [50.98911977,  0.49509856,  0.09341481],
        [47.45092606,  0.48271688,  0.08039621]]])
Dimensions without coordinates: x, y, mom_0

In [280]:
sx.zero()
sx.push_vals(xx, xw, 'rec')
sx.values

<xarray.DataArray (x: 2, y: 3, mom_0: 3)>
array([[[53.48505531,  0.45581937,  0.07743162],
        [50.20687552,  0.45539711,  0.07584982],
        [48.30067994,  0.50679475,  0.10048263]],

       [[47.56845298,  0.4985016 ,  0.09197212],
        [50.98911977,  0.49509856,  0.09341481],
        [47.45092606,  0.48271688,  0.08039621]]])
Dimensions without coordinates: x, y, mom_0

In [247]:
y = xcentral.xStatsAccum.from_data(sx.values)

In [249]:
sx.zeros_like().dims

('x', 'y', 'mom_0')

In [233]:
y.zero()

In [235]:
y

array([[[49.22943366,  0.4538174 ,  0.07525102],
        [46.75869376,  0.48972462,  0.0779285 ],
        [49.10031228,  0.53943114,  0.07121535]],

       [[51.10541143,  0.51031804,  0.08208721],
        [48.31807914,  0.5465915 ,  0.09459712],
        [47.77747568,  0.53468871,  0.07876658]]])

In [218]:
s.reduce(1).values

array([[1.45088440e+02, 4.94362576e-01, 7.60002811e-02],
       [1.47200966e+02, 5.30134712e-01, 8.53477143e-02]])

In [217]:
sx.reduce('y').values

<xarray.DataArray (dim_0: 2, mom_0: 3)>
array([[1.45088440e+02, 4.94362576e-01, 7.60002811e-02],
       [1.47200966e+02, 5.30134712e-01, 8.53477143e-02]])
Dimensions without coordinates: dim_0, mom_0

In [205]:
sx.reduce('x').values

<xarray.DataArray (mom_0: 3)>
array([4.97768848e+02, 5.21029127e-01, 8.08981677e-02])
Dimensions without coordinates: mom_0

In [176]:
xt, target = sx.check_vals(xr.DataArray(x), 'vals')
wt = sx.check_weights(None, target)

In [146]:
xt, target = sx.check_vals(x, 'vals')
wt = sx.check_weights(None, target)

In [147]:
wt

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [122]:
sx.check_vals(xr.DataArray(x), 'vals')

(array([0.93762685, 0.95373213, 0.10187988, 0.00856251, 0.55102326,
        0.55852216, 0.1556304 , 0.94278535, 0.11314863, 0.8170085 ,
        0.78181996, 0.51079837, 0.34058991, 0.56065031, 0.0896993 ,
        0.64889092, 0.50662534, 0.07598245, 0.21797065, 0.58597349,
        0.75889875, 0.43408391, 0.51614074, 0.7644486 , 0.67290985,
        0.01997744, 0.15495179, 0.29068652, 0.30788028, 0.5756541 ,
        0.38161492, 0.77020428, 0.86525487, 0.53607291, 0.94969254,
        0.03317495, 0.53501417, 0.09155597, 0.72128171, 0.35850155,
        0.49813158, 0.08278045, 0.85230378, 0.60298826, 0.46863708,
        0.8345629 , 0.90892238, 0.62340349, 0.34210648, 0.0914277 ,
        0.92821777, 0.24682454, 0.42500105, 0.37912856, 0.17088346,
        0.70678993, 0.81907017, 0.02036205, 0.19078386, 0.78916538,
        0.48098294, 0.95617631, 0.94323499, 0.15484091, 0.59194874,
        0.13492024, 0.96599008, 0.61897894, 0.052144  , 0.91202061,
        0.57498056, 0.68473804, 0.71250856, 0.36

In [116]:
sx.zero()
sx.push_vals(x)

In [117]:
sx.values

<xarray.DataArray (mom_0: 3)>
array([1.00000000e+02, 5.19525042e-01, 8.80988567e-02])
Dimensions without coordinates: mom_0

In [ ]:
sx.

In [118]:
sx.zero()
sx.push_vals(xr.DataArray(x))

In [119]:
sx.values

<xarray.DataArray (mom_0: 3)>
array([51.95250421,  0.68910081,  0.05426992])
Dimensions without coordinates: mom_0

In [109]:
s.push_vals(x)

In [108]:
sx.values

<xarray.DataArray (mom_0: 3)>
array([51.95250421,  0.68910081,  0.05426992])
Dimensions without coordinates: mom_0

In [110]:
s.values

array([1.00000000e+02, 5.19525042e-01, 8.80988567e-02])

In [54]:
sx.push_vals(x)

In [55]:
s.push_vals(x)

In [63]:
s = central.StatsAccum.from_vals(x, w=w)

In [76]:
reload(xcentral)

<module 'cmomy.xcentral' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/xcentral.py'>

In [77]:
sx = xcentral.xStatsAccum.from_vals(xr.DataArray(x), xr.DataArray(w))

ValueError: (0,) must be a permuted list of ('dim_0',), unless `...` is included

In [70]:
xr.DataArray(x).dims

('dim_0',)

In [68]:
sx[0]

{'dims': (),
 'attrs': {},
 'coords': Coordinates:
     *empty*,
 'indexes': ,
 'name': None,
 'moments_dims': None}

In [60]:
s.values

array([1.00000000e+02, 4.73301413e-01, 7.41918339e-02])

In [61]:
sx.values

<xarray.DataArray (mom_0: 3)>
array([1.00000000e+02, 4.73301413e-01, 7.41918339e-02])
Dimensions without coordinates: mom_0

In [33]:
for i in range(100):
    xx = np.random.rand()
    s.push_val(xx)
    sx.push_val(xx)
    

In [40]:
s.values

array([1.00000000e+02, 5.17707754e-01, 7.57878542e-02])

In [41]:
sx.values

<xarray.DataArray (mom_0: 3)>
array([1.00000000e+02, 5.17707754e-01, 7.57878542e-02])
Dimensions without coordinates: mom_0

In [9]:
da = xr.DataArray(np.random.rand(2,3,4))

In [12]:
np.asarray(1)

array(1)

In [15]:
da.sizes

Frozen({'dim_0': 2, 'dim_1': 3, 'dim_2': 4})

In [13]:
da.broadcast_like(xr.DataArray(1))

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 4)>
array([[[0.60827182, 0.30997215, 0.97231283, 0.04206586],
        [0.50261936, 0.85710358, 0.06431332, 0.19017095],
        [0.03990211, 0.82856934, 0.07980946, 0.68861531]],

       [[0.71605623, 0.60094729, 0.94613202, 0.25491901],
        [0.99192776, 0.19699256, 0.50049614, 0.56900732],
        [0.82435319, 0.15914934, 0.61873159, 0.98679198]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [10]:
da * np.zeros(4)

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 4)>
array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [104]:
def _get_cmom_single(w, x, moments, axis=0, last=True):
    wsum = w.sum(axis)
    wsum_inv = 1.0 / wsum

    # get moments
    xave = (w * x).sum(axis) * wsum_inv
    data = [wsum, xave]
    dx = x - xave

    for n in range(2, moments + 1):
        y = (w * dx ** n).sum(axis) * wsum_inv
        data.append(y)

    data = np.array(data)

    if last:
        np.moveaxis(data, 0, -1)

    return data

def _get_data_single(nrec=100, weighted=False):
    x = np.random.rand(nrec)
    if weighted is None:
        w = None
    elif weighted:
        w = np.random.rand(nrec)
    else:
        w = np.ones_like(x)
    return w, x



def test_stats(nrec, moments, weighted):
    # unweighted
    w, x = _get_data_single(nrec, weighted)
    dataA = _get_cmom_single(w, x, moments)

    data = central.central_moments(x, moments, w)
    np.testing.assert_allclose(data, dataA)

    splits = [len(x) // 3, len(x) // 3 * 2]

    ws = np.split(w, splits)
    xs = np.split(x, splits)

    datas = []
    for ww, xx in zip(ws, xs):
        datas.append(_get_cmom_single(ww, xx, moments))

    datas = np.array(datas)

    # factory

    s = central.StatsAccum.from_datas(datas, moments=moments)
    np.testing.assert_allclose(s.data, data)

    # pushs
    s = central.StatsAccum(moments=moments)

    for d in datas:
        s.push_stat(a=d[1], v=d[2:], w=d[0])
    np.testing.assert_allclose(s.data, data)

    s.zero()
#    return s, datas
    s.push_stats(a=datas[:, 1], v=datas[:, 2:], w=datas[:, 0])
    np.testing.assert_allclose(s.data, data)

    s.zero()
    s.push_datas(datas)
    np.testing.assert_allclose(s.data, data)

    # addition
    S = [central.StatsAccum.from_data(d, moments=moments) for d in datas]
    out = S[0]
    for s in S[1:]:
        out = out + s
    np.testing.assert_allclose(out.data, data)

    out = sum(S, central.StatsAccum(moments=moments))
    np.testing.assert_allclose(out.data, data)

    out = central.StatsAccum(moments=moments)
    for s in S:
        out += s
    np.testing.assert_allclose(out.data, data)

    # subtraction
    out = S[0] + S[1] - S[0]
    np.testing.assert_allclose(out.data, datas[1])

    # iadd/isub
    out = central.StatsAccum(moments=moments)
    out += S[0]
    np.testing.assert_allclose(out.data, S[0].data)

    out += S[1]
    np.testing.assert_allclose(out.data, (S[0] + S[1]).data)

    out -= S[0]
    np.testing.assert_allclose(out.data, S[1].data)

    # mult
    out1 = S[0] * 2
    out2 = S[0] + S[0]
    np.testing.assert_allclose(out1.data, out2.data)

    # imul
    out = central.StatsAccum.from_vals(xs[0], ws[0], moments=moments)
    out *= 2
    np.testing.assert_allclose(out.data, (S[0] + S[0]).data)



In [105]:
reload(central)

<module 'cmomy.central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/central.py'>

In [109]:
x = np.random.rand(100, 2, 3)

In [128]:
da = xr.DataArray(x)

In [129]:
da.dims

('dim_0', 'dim_1', 'dim_2')

In [110]:
s = central.StatsAccum.from_vals(x)

In [114]:
s.push_vals(np.random.rand(20,2, 3), w=1)

In [116]:
s.zero()

In [121]:
s.moments

(2,)

In [122]:
s.push_stat(np.random.rand(2,3), v=np.random.rand(2,3,1), w=1.0)

In [125]:
s.push_stats(np.random.rand(2,10,3), v=1.0, w=np.random.rand(10), axis=1)

In [118]:
s.values

array([[[1.00000000e+00, 8.11024292e-01, 1.00000000e+00],
        [1.00000000e+00, 6.66807975e-01, 1.00000000e+00],
        [1.00000000e+00, 7.17184867e-04, 1.00000000e+00]],

       [[1.00000000e+00, 7.94817538e-01, 1.00000000e+00],
        [1.00000000e+00, 4.02544561e-01, 1.00000000e+00],
        [1.00000000e+00, 2.14858253e-01, 1.00000000e+00]]])

In [31]:
np.array([s.data]*2).shape

(2, 2, 3, 3)

In [33]:
central.StatsAccum.from_datas(np.array([s.data]*2))

In [29]:
central.StatsAccum.from_data(s.data).dat

False

In [42]:
da = xr.DataArray(s.data)

In [89]:
da

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [95]:
dda = (
    da
    .assign_coords(dim_0=np.arange(2),
                   dim_1=np.arange(3),
                   dim_2=np.arange(3)
                  )
)

In [98]:
dda.sel(**{'dim_0':0})

<xarray.DataArray (dim_1: 3, dim_2: 3)>
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
Coordinates:
    dim_0    int64 0
  * dim_1    (dim_1) int64 0 1 2
  * dim_2    (dim_2) int64 0 1 2

In [ ]:
xr.ones_like

In [62]:
xr.zeros_like(da)

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [85]:
da.coords

Coordinates:
    *empty*

In [ ]:
da.broadcast_like

In [77]:
n = da.copy()

In [78]:
n[...] = 1

In [84]:
np.asanyarray(da)

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [82]:
n

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [69]:
da.get_axis_num('dim_2')

2

In [68]:
da.transpose('dim_2',...).broadcast_like(da)

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [67]:
xr.broadcast(da, da.transpose('dim_2',...))

(<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
 array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
 Dimensions without coordinates: dim_0, dim_1, dim_2,
 <xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 3)>
 array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
 Dimensions without coordinates: dim_0, dim_1, dim_2)

In [65]:
xr.broadcast(da, da.transpose('dim_2','dims_1','dim_0'))

ValueError: ('dim_2', 'dims_1', 'dim_0') must be a permuted list of ('dim_0', 'dim_1', 'dim_2'), unless `...` is included

In [57]:
da.dims[:-1]

('dim_0', 'dim_1')

In [55]:
da[...,0]

<xarray.DataArray (dim_0: 2, dim_1: 3)>
array([[0., 0., 0.],
       [0., 0., 0.]])
Dimensions without coordinates: dim_0, dim_1

In [53]:
da.indexes

xarray.core.indexes.Indexes

In [47]:
da.get_axis_num('dim_0')

0

In [161]:
c = central.StatsAccumCov.from_vals(x, x)

In [137]:
np.sum(s, axis=0, keepdims=True)

(<cmomy.central.StatsAccum at 0x7f8532024510>,
 array([[[2.00000000e+02, 9.86753753e-01, 1.46854042e-01],
         [2.00000000e+02, 1.02552571e+00, 1.55524850e-01],
         [2.00000000e+02, 9.78268499e-01, 1.79736079e-01]]]),
 None)

In [16]:
s._data

array([[[1.00000000e+02, 4.79796944e-01, 8.54884879e-02],
        [1.00000000e+02, 5.09763042e-01, 7.18994589e-02],
        [1.00000000e+02, 4.56756545e-01, 9.00735834e-02]],

       [[1.00000000e+02, 4.96641179e-01, 8.39759371e-02],
        [1.00000000e+02, 4.93719663e-01, 8.45732967e-02],
        [1.00000000e+02, 4.97347949e-01, 7.50247842e-02]]])

In [14]:
'{}({})'.format(type(s), rs._data)

"<class 'cmomy.central.StatsAccum'>([[[1.00000000e+02 4.79796944e-01 8.54884879e-02]\n  [1.00000000e+02 5.09763042e-01 7.18994589e-02]\n  [1.00000000e+02 4.56756545e-01 9.00735834e-02]]\n\n [[1.00000000e+02 4.96641179e-01 8.39759371e-02]\n  [1.00000000e+02 4.93719663e-01 8.45732967e-02]\n  [1.00000000e+02 4.97347949e-01 7.50247842e-02]]])"

In [164]:
c._moments_len

2

In [163]:
c.__dict__.keys()

dict_keys(['moments', 'shape', '_push_val', '_push_vals', '_push_data', '_push_datas', '_cache', '_data', '_data_flat'])

In [162]:
s.__dict__.keys()

dict_keys(['moments', 'shape', '_push_val', '_push_vals', '_push_stat', '_push_stats', '_push_data', '_push_datas', '_cache', '_data', '_data_flat'])

In [ ]:
np.asarray()

In [129]:
da = xr.DataArray(s.data)

In [143]:
a = x.astype(np.float32)

In [158]:
o = np.asarray(a, dtype=np.float)

In [159]:
o is a

False

In [139]:
np.array(da)

array([[[1.00000000e+02, 4.97045811e-01, 9.49196301e-02],
        [1.00000000e+02, 5.33672019e-01, 9.03008228e-02],
        [1.00000000e+02, 5.02405390e-01, 7.87018413e-02]],

       [[1.00000000e+02, 5.13801148e-01, 7.95859473e-02],
        [1.00000000e+02, 5.20231620e-01, 7.62445599e-02],
        [1.00000000e+02, 5.14365816e-01, 8.45126265e-02]]])

In [ ]:
np.asarray()

In [138]:
np.asarray(da)

array([[[1.00000000e+02, 4.97045811e-01, 9.49196301e-02],
        [1.00000000e+02, 5.33672019e-01, 9.03008228e-02],
        [1.00000000e+02, 5.02405390e-01, 7.87018413e-02]],

       [[1.00000000e+02, 5.13801148e-01, 7.95859473e-02],
        [1.00000000e+02, 5.20231620e-01, 7.62445599e-02],
        [1.00000000e+02, 5.14365816e-01, 8.45126265e-02]]])

In [127]:
np.moveaxis(s, 0, 1)

array([[[1.00000000e+02, 4.97045811e-01, 9.49196301e-02],
        [1.00000000e+02, 5.13801148e-01, 7.95859473e-02]],

       [[1.00000000e+02, 5.33672019e-01, 9.03008228e-02],
        [1.00000000e+02, 5.20231620e-01, 7.62445599e-02]],

       [[1.00000000e+02, 5.02405390e-01, 7.87018413e-02],
        [1.00000000e+02, 5.14365816e-01, 8.45126265e-02]]])

In [110]:
# huge resample

In [111]:
x = np.random.rand(1000)

In [98]:
nrep = 10000

In [99]:
idx = np.random.choice(len(x), (nrep, len(x)), replace=True)

In [105]:
out = central.central_moments(x[idx], moments=3, axis=1)

freq = central.randsamp_freq(indices=idx)

s = central.StatsAccum.from_resample_vals(x, freq, moments=3)

In [108]:
np.asarray(s)

array([[ 1.00000000e+03,  4.96193323e-01,  7.95093035e-02,
         1.86688590e-04],
       [ 1.00000000e+03,  4.83232909e-01,  8.77191301e-02,
         2.34503656e-04],
       [ 1.00000000e+03,  4.86318703e-01,  8.79459487e-02,
         5.73834077e-04],
       ...,
       [ 1.00000000e+03,  5.03417804e-01,  8.03625554e-02,
        -2.23532273e-04],
       [ 1.00000000e+03,  4.98975666e-01,  8.68628949e-02,
        -1.47378615e-03],
       [ 1.00000000e+03,  5.05772017e-01,  8.32541233e-02,
        -1.46964042e-03]])

In [101]:
np.testing.assert_allclose(s.data, out, atol=1e-10)

In [102]:
%timeit -n 1 -r 1 out = central.central_moments(x[idx], moments=3, axis=1)

%timeit -n 1 -r 1 freq = central.randsamp_freq(indices=idx)

%timeit -n 1 -r 1 s = central.StatsAccum.from_resample_vals(x, freq, moments=3)

577 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
44.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
147 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [104]:
%timeit -n 1 -r 1 s = central.StatsAccum.from_resample_vals(x, freq, moments=3, parallel=True)

37.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [63]:
out.shape

(100, 4)

In [22]:
# raw/central

In [44]:
x = np.random.rand(1000, 100)

In [45]:
s = central.StatsAccum.from_vals(x)
a = statsarray.StatsArray.from_accum(s, axis=0)

In [46]:
s0 = accumulator.StatsAccumVec.from_vals(x)
a0 = accumulator.StatsArray.from_datas(s0.data)

In [47]:
%timeit -n 10 -r 3 s = central.StatsAccum.from_vals(x)
%timeit -n 10 -r 3 s0 = accumulator.StatsAccumVec.from_vals(x)

1.96 ms ± 77.6 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
1.48 ms ± 131 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [32]:
np.testing.assert_allclose(a0.mean(),a.mean())

In [33]:
np.testing.assert_allclose(a0.var(), a.var())

In [34]:
nrep= 200
ndat = len(a)

In [35]:
idx = np.random.choice(nrep, size=(nrep, ndat), replace=True)
freq = central.randsamp_freq(indices=idx)

In [17]:
r0 = a0.resample_and_reduce(freq.T)

r = a.resample_and_reduce(freq)

In [18]:
np.testing.assert_allclose(r.data, r0.data)

In [19]:
np.testing.assert_allclose(r.cumdata, r0.cumdata)

In [20]:
%timeit -n 1 -r 1 r0 = a0.resample_and_reduce(freq.T)

%timeit -n 1 -r 1 r = a.resample_and_reduce(freq)

490 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
810 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [278]:
r0.mean_SEM()

0.0018966836826551947

In [279]:
r.mean_SEM()

0.0018966836826551947

In [282]:
a = r.to_xarray()

In [283]:
b = r0.to_xarray()

In [287]:
rr = central.StatsArray.from_xarray(a)

In [288]:
rr0 = accumulator.StatsArray.from_xarray(b)

In [290]:
np.testing.assert_allclose(r.data, rr.data)

In [291]:
np.testing.assert_allclose(r0.data, rr0.data)